# Four Wave Mixing Case

In [46]:
from sympy import *
# from sympy import (
#     symbols,
#     Function,
#     IndexedBase,
#     Wild,
#     Eq
# )

# kth order derivatives of Weierstrass P
# from wpk import wpk, wzk, wsk, run_tests

# The package containing mpmath expressions for Weierstrass elliptic functions
from weierstrass_modified import Weierstrass
# from mpmath import exp as mpexp

# Numeric solutions to diff eqs
from numpy import (
    # linspace, absolute, angle, square, real, imag, 
    conj, 
    # array as arraynp, 
    # concatenate
)
# from numpy import vectorize as np_vectorize # not to get confused with vectorise in other packages
# import scipy.integrate
# import matplotlib.pyplot as plt
# from random import random

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all' # type: ignore

we = Weierstrass()

# -- Symbols --

(x, y, z, t, x0, y0, z0, z1, t0, Z, g2, g3, m, n, l, k, i, j, M, N, C, n2, T) = symbols(
    '''x, y, z, t, x0, y0, z0, z1, t0, Z, g2, g3, m, n, l, k, i, j, M, N, C, n2, T'''
)
(delta, nu, Aeff) = symbols(
    '''delta, nu, Aeff'''
)

# -- Functions --

pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass Zeta function
sigma = Function('sigma') # Weierstrass Sigma function
wp = Function('wp') # Weierstrass P function
wpp = Function(r"\wp'") # Derivative of Weierstrass P function
zeta = Function('zeta') # Weierstrass Zeta function

rhop = Function(r"\rho'")
Delta = Function('Delta')
rho = Function('rho')
kappa = Function('kappa')
phi = Function('phi')
h = Function('h')
q = Function('q')
s = Function('s')
u = Function('u')
v = Function('v')
w = Function('w')
chi = Function('chi')
P = Function('P') # Polynomial
Q = Function('Q') # Polynomial
R = Function('R') # Polynomial

U = Function('U')
V = Function('V')
U0 = Function('U0')
V0 = Function('V0')
W = Function('W')
H = Function('H')

wp = Function('wp')
wp22 = Function('wp22')
wp21 = Function('wp21')
wp11 = Function('wp11')

A = Function('A')
Ac = Function('Ac')
A1 = Function('A1')
A2 = Function('A2')
A3 = Function('A3')
A4 = Function('A4')
Ac1 = Function('Ac1')
Ac2 = Function('Ac2')
Ac3 = Function('Ac3')
Ac4 = Function('Ac4')

B = Function('B')
Bc = Function('Bc')
B1 = Function('B1')
B2 = Function('B2')
B3 = Function('B3')
B4 = Function('B4')
Bc1 = Function('Bc1')
Bc2 = Function('Bc2')
Bc3 = Function('Bc3')
Bc4 = Function('Bc4')

# -- Indexed Symbols --

Omega = IndexedBase('Omega')
Xi = IndexedBase('Xi')
f = IndexedBase('f')
fc = IndexedBase('fc')
F = IndexedBase('F')
r = IndexedBase('r')
gamma = IndexedBase('gamma')
mu = IndexedBase('mu')
nu = IndexedBase('nu')
xi = IndexedBase('xi')
theta = IndexedBase('theta')
X = IndexedBase('X')
Y = IndexedBase('Y')
a = IndexedBase('a')
b = IndexedBase('b')
c = IndexedBase('c')
d = IndexedBase('d')
p = IndexedBase('p')
G = IndexedBase('G')
psi = IndexedBase('psi')
upsilon = IndexedBase('upsilon')
epsilon = IndexedBase('epsilon')
omega = IndexedBase('omega')
alpha = IndexedBase('alpha')
beta = IndexedBase('beta')
K = IndexedBase('K')
lam = IndexedBase('lambda')
Lam = IndexedBase('Lambda')

wild = Wild('*')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
def bs(eq, func):
    """Apply func to both sides"""
    return Eq(func(eq.lhs), func(eq.rhs))

## Elliptic function identities

In [30]:
sigma_p_identity = Eq(
    wp(y, g2, g3) - wp(x, g2, g3),
    sigma(x + y, g2, g3) * sigma(x - y, g2, g3) / (sigma(x, g2, g3) ** 2 * sigma(y, g2, g3) ** 2) 
)
pw_to_zw_identity = Eq(
    (wpp(z,g2,g3) - wpp(x,g2,g3))/(wp(z,g2,g3) - wp(x,g2,g3))/2,
    zeta(z + x,g2, g3) - zeta(z,g2, g3) - zeta(x,g2, g3)
)
pw_to_zw_identity_one_sided = Eq(
    wpp(z, g2, g3)/(wp(x, g2, g3) - wp(z, g2, g3)),
    2*zeta(z, g2, g3) - zeta(-x + z, g2, g3) - zeta(x + z, g2, g3)
)

zw_dlog_sigma_identity = Eq(zeta(z - x, g2, g3), Derivative(log(sigma(z - x, g2, g3)), z))
pw_to_dlog_sigma_identity = pw_to_zw_identity.subs([
    zw_dlog_sigma_identity.subs(x,-x).args,
    zw_dlog_sigma_identity.subs(x,0).args,
])
pw_to_dlog_sigma_identity_b = pw_to_dlog_sigma_identity.subs(x,-x).subs([
    (wp(-x,g2,g3), wp(x,g2,g3)), (wpp(-x,g2,g3), -wpp(x,g2,g3)), (zeta(-x, g2,g3), -zeta(x, g2,g3))
])

pw_addition_id = Eq(
    wp(x + y, g2, g3),
    -wp(x, g2, g3) - wp(y, g2, g3) + (wpp(x, g2, g3) - wpp(y, g2, g3))**2/(4*(wp(x, g2, g3) - wp(y, g2, g3))**2)
)

pwp_sigma_dbl_ratio = Eq(wpp(z,g2,g3), - sigma(2*z,g2,g3)/sigma(z,g2,g3)**4)
quasi_period_sigma_b = Eq(
    sigma(2*m*omega[3] + 2*n*omega[1] + z, g2, g3),
    (-1)**(m*n + m + n)*sigma(z, g2, g3)*
    exp(2*(m*omega[3] + n*omega[1] + z)*zeta(m*omega[3] + n*omega[1], g2, g3))
)

pw_addition_id_omega_g2 = Eq(
    wp(z + omega[i], g2, g3) - wp(omega[i], g2, g3),
    (-g2/4 + 3*wp(omega[i], g2, g3)**2)/(wp(z, g2, g3) - wp(omega[i], g2, g3))
)

pwp_xy_addition = Eq(wpp(x + y, g2, g3),
    -(wpp(x, g2, g3) + wpp(y, g2, g3))/2 + 
    3*(wp(x, g2, g3) + wp(y, g2, g3))*(wpp(x, g2, g3) - wpp(y, g2, g3))/(2*(wp(x, g2, g3) - wp(y, g2, g3)))
    - (wpp(x, g2, g3) - wpp(y, g2, g3))**3/(4*(wp(x, g2, g3) - wp(y, g2, g3))**3)
)

sigma_p_identity
pw_to_zw_identity
pw_to_zw_identity_one_sided
zw_dlog_sigma_identity
pw_to_dlog_sigma_identity_b
pw_addition_id
pwp_sigma_dbl_ratio
quasi_period_sigma_b
pwp_xy_addition

Eq(-wp(x, g2, g3) + wp(y, g2, g3), sigma(x - y, g2, g3)*sigma(x + y, g2, g3)/(sigma(x, g2, g3)**2*sigma(y, g2, g3)**2))

Eq((-\wp'(x, g2, g3) + \wp'(z, g2, g3))/(2*(-wp(x, g2, g3) + wp(z, g2, g3))), -zeta(x, g2, g3) - zeta(z, g2, g3) + zeta(x + z, g2, g3))

Eq(\wp'(z, g2, g3)/(wp(x, g2, g3) - wp(z, g2, g3)), 2*zeta(z, g2, g3) - zeta(-x + z, g2, g3) - zeta(x + z, g2, g3))

Eq(zeta(-x + z, g2, g3), Derivative(log(sigma(-x + z, g2, g3)), z))

Eq((\wp'(x, g2, g3) + \wp'(z, g2, g3))/(2*(-wp(x, g2, g3) + wp(z, g2, g3))), zeta(x, g2, g3) - Derivative(log(sigma(z, g2, g3)), z) + Derivative(log(sigma(-x + z, g2, g3)), z))

Eq(wp(x + y, g2, g3), (\wp'(x, g2, g3) - \wp'(y, g2, g3))**2/(4*(wp(x, g2, g3) - wp(y, g2, g3))**2) - wp(x, g2, g3) - wp(y, g2, g3))

Eq(\wp'(z, g2, g3), -sigma(2*z, g2, g3)/sigma(z, g2, g3)**4)

Eq(sigma(2*m*omega[3] + 2*n*omega[1] + z, g2, g3), (-1)**(m*n + m + n)*sigma(z, g2, g3)*exp((2*m*omega[3] + 2*n*omega[1] + 2*z)*zeta(m*omega[3] + n*omega[1], g2, g3)))

Eq(\wp'(x + y, g2, g3), -(\wp'(x, g2, g3) - \wp'(y, g2, g3))**3/(4*(wp(x, g2, g3) - wp(y, g2, g3))**3) + (\wp'(x, g2, g3) - \wp'(y, g2, g3))*(3*wp(x, g2, g3) + 3*wp(y, g2, g3))/(2*wp(x, g2, g3) - 2*wp(y, g2, g3)) - \wp'(x, g2, g3)/2 - \wp'(y, g2, g3)/2)

In [31]:
pw_addition_id_omega = (
    pw_addition_id
    .subs([
        (x,z), 
        (y,omega[i]), 
        (wpp(omega[i],g2,g3),0),
        (
            wpp(z,g2,g3)**2,
            (wp(z,g2,g3) - wp(omega[i],g2,g3))*
            (wp(z,g2,g3) - wp(omega[j],g2,g3))*
            (4*wp(z,g2,g3) - 4*wp(omega[k],g2,g3))
        )
    ])
)
pw_addition_id_omega = Eq(
    pw_addition_id_omega.lhs - wp(omega[i],g2,g3), 
    (
        apart(
            pw_addition_id_omega.rhs.subs(wp(z,g2,g3),X)
            ,
            X
        ).subs(X, wp(z,g2,g3)) 
        - wp(omega[i],g2,g3)
    ).subs(wp(omega[i],g2,g3) + wp(omega[j],g2,g3) + wp(omega[k],g2,g3), 0)
)
omega_i_j_k_a = Eq(
    fraction(pw_addition_id_omega.rhs)[0],
    fraction(pw_addition_id_omega.rhs)[0].expand()
)
omega_i_j_k_b = Eq(
    4*
    (wp(z, g2, g3) - wp(omega[i], g2, g3))*
    (wp(z, g2, g3) - wp(omega[j], g2, g3))*
    (wp(z, g2, g3) - wp(omega[k], g2, g3)),
    (
        4*
        (wp(z, g2, g3) - wp(omega[i], g2, g3))*
        (wp(z, g2, g3) - wp(omega[j], g2, g3))*
        (wp(z, g2, g3) - wp(omega[k], g2, g3))
    ).expand().collect(wp(z,g2,g3), factor)
    
)
g2_omega_i_j_k = Eq(g2, -omega_i_j_k_b.rhs.coeff(wp(z,g2,g3)))
wp_omega_i_sqrd_j_k = Eq((wp(omega[i],g2,g3)*omega_i_j_k_b.rhs.coeff(wp(z,g2,g3),2)).expand()/4, 0)
wp_omega_i_sqrd_j_k =  Eq(wp(omega[i],g2,g3)**2, solve(wp_omega_i_sqrd_j_k, wp(omega[i],g2,g3)**2)[0].expand())

g2_min_wp_omega_i_omega_j_k = Eq(
    g2_omega_i_j_k.lhs/4 - wp_omega_i_sqrd_j_k.lhs,
    (g2_omega_i_j_k.rhs/4 - wp_omega_i_sqrd_j_k.rhs).expand()
)

omega_i_j_k_c = Eq(
    omega_i_j_k_a.lhs, 
    omega_i_j_k_a.rhs
    + wp_omega_i_sqrd_j_k.lhs - wp_omega_i_sqrd_j_k.rhs
    - g2_min_wp_omega_i_omega_j_k.lhs + g2_min_wp_omega_i_omega_j_k.rhs
)

pw_addition_id_omega_g2 = pw_addition_id_omega.subs(*omega_i_j_k_c.args).simplify()

# omega_i_j_k_a
# omega_i_j_k_b
# g2_omega_i_j_k
# wp_omega_i_sqrd_j_k
# g2_min_wp_omega_i_omega_j_k
# omega_i_j_k_c

pw_addition_id_omega_g2

Eq(wp(z + omega[i], g2, g3) - wp(omega[i], g2, g3), (-g2/4 + 3*wp(omega[i], g2, g3)**2)/(wp(z, g2, g3) - wp(omega[i], g2, g3)))

## The FWM system from Agrawal

*Theory of Four-Wave Mixing, p371, Nonlinear Fiber Optics 4th Edition, Govind P. Agrawal*

The book references [this paper](https://ieeexplore.ieee.org/document/1071660) but it is not open access.

In [32]:
dA1 = Eq(
    diff(A1(z),z), 
    I*n2*omega[1]/c*(
        (
            f[1,1]*A1(z)*Ac1(z) + 
            2*(f[1,2]*A2(z)*Ac2(z) + f[1,3]*A3(z)*Ac3(z) + f[1,4]*A4(z)*Ac4(z))
        )*A1(z) +
        2*f[1,2,3,4]*Ac2(z)*A3(z)*A4(z)*exp(I*Delta(k)*z)
    )
)
dA2 = Eq(
    diff(A2(z),z), 
    I*n2*omega[2]/c*(
        (
            f[2,2]*A2(z)*Ac2(z) + 
            2*(f[2,1]*A1(z)*Ac1(z) + f[2,3]*A3(z)*Ac3(z) + f[2,4]*A4(z)*Ac4(z))
        )*A2(z) +
        2*f[2,1,3,4]*Ac1(z)*A3(z)*A4(z)*exp(I*Delta(k)*z)
    )
)
dA3 = Eq(
    diff(A3(z),z), 
    I*n2*omega[3]/c*(
        (
            f[3,3]*A3(z)*Ac3(z) + 
            2*(f[3,1]*A1(z)*Ac1(z) + f[3,2]*A2(z)*Ac2(z) + f[3,4]*A4(z)*Ac4(z))
        )*A3(z) +
        2*f[3,4,1,2]*A1(z)*A2(z)*Ac4(z)*exp(-I*Delta(k)*z)
    )
)
dA4 = Eq(
    diff(A4(z),z), 
    I*n2*omega[4]/c*(
        (
            f[4,4]*A4(z)*Ac4(z) + 
            2*(f[4,1]*A1(z)*Ac1(z) + f[4,2]*A2(z)*Ac2(z) + f[4,3]*A3(z)*Ac3(z))
        )*A4(z) +
        2*f[4,3,1,2]*A1(z)*A2(z)*Ac3(z)*exp(-I*Delta(k)*z)
    )
)

dAc1 = Eq(
    diff(Ac1(z),z), 
    -I*n2*omega[1]/c*(
        (
            f[1,1]*A1(z)*Ac1(z) + 
            2*(f[1,2]*A2(z)*Ac2(z) + f[1,3]*A3(z)*Ac3(z) + f[1,4]*A4(z)*Ac4(z))
        )*Ac1(z) +
        2*conjugate(f[1,2,3,4])*A2(z)*Ac3(z)*Ac4(z)*exp(-I*Delta(k)*z)
    )
)
dAc2 = Eq(
    diff(Ac2(z),z), 
    -I*n2*omega[2]/c*(
        (
            f[2,2]*A2(z)*Ac2(z) + 
            2*(f[2,1]*A1(z)*Ac1(z) + f[2,3]*A3(z)*Ac3(z) + f[2,4]*A4(z)*Ac4(z))
        )*Ac2(z) +
        2*conjugate(f[2,1,3,4])*A1(z)*Ac3(z)*Ac4(z)*exp(-I*Delta(k)*z)
    )
)
dAc3 = Eq(
    diff(Ac3(z),z), 
    -I*n2*omega[3]/c*(
        (
            f[3,3]*A3(z)*Ac3(z) + 
            2*(f[3,1]*A1(z)*Ac1(z) + f[3,2]*A2(z)*Ac2(z) + f[3,4]*A4(z)*Ac4(z))
        )*Ac3(z) +
        2*conjugate(f[3,4,1,2])*Ac1(z)*Ac2(z)*A4(z)*exp(I*Delta(k)*z)
    )
)
dAc4 = Eq(
    diff(Ac4(z),z), 
    -I*n2*omega[4]/c*(
        (
            f[4,4]*A4(z)*Ac4(z) + 
            2*(f[4,1]*A1(z)*Ac1(z) + f[4,2]*A2(z)*Ac2(z) + f[4,3]*A3(z)*Ac3(z))
        )*Ac4(z) +
        2*conjugate(f[4,3,1,2])*Ac1(z)*Ac2(z)*A3(z)*exp(I*Delta(k)*z)
    )
)
wave_mixing_fs = [f[1,2,3,4], f[2,1,3,4], f[3,4,1,2], f[4,3,1,2]]
wave_mixing_fcs = [conjugate(f[1,2,3,4]), conjugate(f[2,1,3,4]), conjugate(f[3,4,1,2]), conjugate(f[4,3,1,2])]

dAs = [Eq(_eq.lhs, _eq.rhs.expand().collect(wave_mixing_fs, factor)) for _eq in [dA1, dA2, dA3, dA4]]
dAcs = [Eq(_eq.lhs, _eq.rhs.expand().collect(wave_mixing_fcs, factor)) for _eq in [dAc1, dAc2, dAc3, dAc4]]

delta_k = Eq(Delta(k), beta[3] + beta[4] - beta[1] - beta[2])
beta_4_delta_k_sub = [(beta[4], solve(delta_k, beta[4])[0])]
omega_k = Eq(omega[3] + omega[4], omega[1] + omega[2])
    
for _eq in dAs:
    _eq
    
delta_k
omega_k

Eq(Derivative(A1(z), z), I*n2*(A1(z)*Ac1(z)*f[1, 1] + 2*A2(z)*Ac2(z)*f[1, 2] + 2*A3(z)*Ac3(z)*f[1, 3] + 2*A4(z)*Ac4(z)*f[1, 4])*A1(z)*omega[1]/c + 2*I*n2*A3(z)*A4(z)*Ac2(z)*exp(I*z*Delta(k))*f[1, 2, 3, 4]*omega[1]/c)

Eq(Derivative(A2(z), z), I*n2*(2*A1(z)*Ac1(z)*f[2, 1] + A2(z)*Ac2(z)*f[2, 2] + 2*A3(z)*Ac3(z)*f[2, 3] + 2*A4(z)*Ac4(z)*f[2, 4])*A2(z)*omega[2]/c + 2*I*n2*A3(z)*A4(z)*Ac1(z)*exp(I*z*Delta(k))*f[2, 1, 3, 4]*omega[2]/c)

Eq(Derivative(A3(z), z), I*n2*(2*A1(z)*Ac1(z)*f[3, 1] + 2*A2(z)*Ac2(z)*f[3, 2] + A3(z)*Ac3(z)*f[3, 3] + 2*A4(z)*Ac4(z)*f[3, 4])*A3(z)*omega[3]/c + 2*I*n2*A1(z)*A2(z)*Ac4(z)*exp(-I*z*Delta(k))*f[3, 4, 1, 2]*omega[3]/c)

Eq(Derivative(A4(z), z), I*n2*(2*A1(z)*Ac1(z)*f[4, 1] + 2*A2(z)*Ac2(z)*f[4, 2] + 2*A3(z)*Ac3(z)*f[4, 3] + A4(z)*Ac4(z)*f[4, 4])*A4(z)*omega[4]/c + 2*I*n2*A1(z)*A2(z)*Ac3(z)*exp(-I*z*Delta(k))*f[4, 3, 1, 2]*omega[4]/c)

Eq(Delta(k), -beta[1] - beta[2] + beta[3] + beta[4])

Eq(omega[3] + omega[4], omega[1] + omega[2])

Agrawal states we can make an effective area approximation of modal overlap integrals valid for single mode fibers in which the below holds. However, it is not needed as we can implement a more general transform to achieve the same thing without further approximation. However, it seems the only difference is it discards a complex phase which we account for by rotating the fields.

In [33]:
Eq(f[i,j], 1/Aeff)
Eq(f[i,j,k,l], 1/Aeff)

Eq(f[i, j], 1/Aeff)

Eq(f[i, j, k, l], 1/Aeff)

## Transforming the four wave mixing system to the unified theory format

We want to put the system in the following form for the unified theory, where we can assume $a_{j,k}=a_{k,j}$ without loss of generality:

In [34]:
du_phase_mod_part = (a[j] + Sum(a[j,k]*u(z,mu[k])*v(z,mu[k]), (k,1,4)))*u(z,mu[j])
du_mixing_part = Product(v(z,mu[k]), (k,1,4))/v(z, mu[j])
duj = Eq(diff(u(z,mu[j]),z), -du_phase_mod_part + du_mixing_part)

dv_phase_mod_part = -(a[j] + Sum(a[j,k]*u(z,mu[k])*v(z,mu[k]), (k,1,4)))*v(z,mu[j])
dv_mixing_part = -Product(u(z,mu[k]), (k,1,4))/u(z, mu[j])
dvj = Eq(diff(v(z,mu[j]),z), (-dv_phase_mod_part).factor() + dv_mixing_part)

Quv_eq = Eq(
    Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[2])*v(z, mu[2]), u(z, mu[3])*v(z, mu[3]), u(z, mu[4])*v(z, mu[4])),
    a[0] + Sum(a[j]*u(z, mu[j])*v(z, mu[j]), (j,1,4)) + 
    Sum(a[j,k]/2*u(z, mu[j])*v(z, mu[j])*u(z, mu[k])*v(z, mu[k]), (j,1,4), (k,1,4))
)
Eq(a[j,k], a[k,j])

duj
dvj
Quv_eq

Eq(a[j, k], a[k, j])

Eq(Derivative(u(z, mu[j]), z), -(a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*u(z, mu[j]) + Product(v(z, mu[k]), (k, 1, 4))/v(z, mu[j]))

Eq(Derivative(v(z, mu[j]), z), (a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*v(z, mu[j]) - Product(u(z, mu[k]), (k, 1, 4))/u(z, mu[j]))

Eq(Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[2])*v(z, mu[2]), u(z, mu[3])*v(z, mu[3]), u(z, mu[4])*v(z, mu[4])), a[0] + Sum(u(z, mu[j])*v(z, mu[j])*a[j], (j, 1, 4)) + Sum(u(z, mu[j])*u(z, mu[k])*v(z, mu[j])*v(z, mu[k])*a[j, k]/2, (j, 1, 4), (k, 1, 4)))

We do this by:
- introducing global phase rotations to absorb any phase coming from complex wave mixing modal overlaps (e.g., in case $f_{1,2,3,4}$ etc. are complex)
- then rescaling with $r_j$ parameters to make all wave mixing coefficients equal to 1
- reparameterising to simplify the way we represent the parameters in the phase modulation polynomial (e.g. to use $a_j, a_{j,k}$)


In order to homegenise the FWM coefficients we introduce the phase rotated fields $B$ and the FWM coefficient phases $\nu$:

In [36]:
AB_subs = [
    (A1(z), B1(z)*exp(I*theta[1])),
    (A2(z), B2(z)*exp(I*theta[2])),
    (A3(z), B3(z)*exp(I*theta[3])),
    (A4(z), B4(z)*exp(I*theta[4])),
    (Ac1(z), Bc1(z)*exp(-I*theta[1])),
    (Ac2(z), Bc2(z)*exp(-I*theta[2])),
    (Ac3(z), Bc3(z)*exp(-I*theta[3])),
    (Ac4(z), Bc4(z)*exp(-I*theta[4])),
]
f_fc_subs = [
    (conjugate(f[1,2,3,4]), Abs(f[1,2,3,4])*exp(-I*nu[1])),
    (conjugate(f[2,1,3,4]), Abs(f[2,1,3,4])*exp(-I*nu[2])),
    (conjugate(f[3,4,1,2]), Abs(f[3,4,1,2])*exp(-I*nu[3])),
    (conjugate(f[4,3,1,2]), Abs(f[4,3,1,2])*exp(-I*nu[4])),
    (f[1,2,3,4], Abs(f[1,2,3,4])*exp(I*nu[1])),
    (f[2,1,3,4], Abs(f[2,1,3,4])*exp(I*nu[2])),
    (f[3,4,1,2], Abs(f[3,4,1,2])*exp(I*nu[3])),
    (f[4,3,1,2], Abs(f[4,3,1,2])*exp(I*nu[4])),
]

for _args in AB_subs[0:4]:
    Eq(*_args)
    
for _args in f_fc_subs[4:8]:
    Eq(*_args)

Eq(A1(z), B1(z)*exp(I*theta[1]))

Eq(A2(z), B2(z)*exp(I*theta[2]))

Eq(A3(z), B3(z)*exp(I*theta[3]))

Eq(A4(z), B4(z)*exp(I*theta[4]))

Eq(f[1, 2, 3, 4], exp(I*nu[1])*Abs(f[1, 2, 3, 4]))

Eq(f[2, 1, 3, 4], exp(I*nu[2])*Abs(f[2, 1, 3, 4]))

Eq(f[3, 4, 1, 2], exp(I*nu[3])*Abs(f[3, 4, 1, 2]))

Eq(f[4, 3, 1, 2], exp(I*nu[4])*Abs(f[4, 3, 1, 2]))

We insert these into the equations of motion and observe that we can cancel the complex parts of the FWM coefficients using $\theta_j$ provided certain relations hold among the $\nu_j$:

In [18]:
dBs_nu = [
    bs(
        bs(
            _eq.subs(AB_subs + f_fc_subs), 
            lambda x: (x*exp(-I*theta[_i+1])).doit()
        ), 
        lambda x: x.simplify().expand().collect(Abs(wave_mixing_fs[_i]), simplify)
    )
    for _i, _eq in enumerate(dAs)
]
dBcs_nu = [
    bs(
        bs(
            _eq.subs(AB_subs + f_fc_subs), 
            lambda x: (x*exp(I*theta[_i+1])).doit()
        ), 
        lambda x: x.simplify().expand().collect(Abs(wave_mixing_fs[_i]), simplify)
    )
    for _i, _eq in enumerate(dAcs)
]

for _eq in dBs_nu:
    _eq
# for _eq in dBcs_nu:
#     _eq

Eq(Derivative(B1(z), z), I*n2*(B1(z)*Bc1(z)*f[1, 1] + 2*B2(z)*Bc2(z)*f[1, 2] + 2*B3(z)*Bc3(z)*f[1, 3] + 2*B4(z)*Bc4(z)*f[1, 4])*B1(z)*omega[1]/c + 2*I*n2*B3(z)*B4(z)*Bc2(z)*exp(I*(z*Delta(k) + nu[1] - theta[1] - theta[2] + theta[3] + theta[4]))*Abs(f[1, 2, 3, 4])*omega[1]/c)

Eq(Derivative(B2(z), z), I*n2*(2*B1(z)*Bc1(z)*f[2, 1] + B2(z)*Bc2(z)*f[2, 2] + 2*B3(z)*Bc3(z)*f[2, 3] + 2*B4(z)*Bc4(z)*f[2, 4])*B2(z)*omega[2]/c + 2*I*n2*B3(z)*B4(z)*Bc1(z)*exp(I*(z*Delta(k) + nu[2] - theta[1] - theta[2] + theta[3] + theta[4]))*Abs(f[2, 1, 3, 4])*omega[2]/c)

Eq(Derivative(B3(z), z), I*n2*(2*B1(z)*Bc1(z)*f[3, 1] + 2*B2(z)*Bc2(z)*f[3, 2] + B3(z)*Bc3(z)*f[3, 3] + 2*B4(z)*Bc4(z)*f[3, 4])*B3(z)*omega[3]/c + 2*I*n2*B1(z)*B2(z)*Bc4(z)*exp(I*(-z*Delta(k) + nu[3] + theta[1] + theta[2] - theta[3] - theta[4]))*Abs(f[3, 4, 1, 2])*omega[3]/c)

Eq(Derivative(B4(z), z), I*n2*(2*B1(z)*Bc1(z)*f[4, 1] + 2*B2(z)*Bc2(z)*f[4, 2] + 2*B3(z)*Bc3(z)*f[4, 3] + B4(z)*Bc4(z)*f[4, 4])*B4(z)*omega[4]/c + 2*I*n2*B1(z)*B2(z)*Bc3(z)*exp(I*(-z*Delta(k) + nu[4] + theta[1] + theta[2] - theta[3] - theta[4]))*Abs(f[4, 3, 1, 2])*omega[4]/c)

Specifically, the following are required:

In [19]:
nu_subs = [
    (nu[1], theta[1] + theta[2] - theta[3] - theta[4]),
    (nu[2], theta[1] + theta[2] - theta[3] - theta[4]),
    (nu[3], -theta[1] - theta[2] + theta[3] + theta[4]),
    (nu[4], -theta[1] - theta[2] + theta[3] + theta[4])
]
for _args in nu_subs:
    Eq(*_args)
    
Eq(nu[2], nu[1])
Eq(nu[3], -nu[1])
Eq(nu[4], -nu[1])

Eq(nu[1], theta[1] + theta[2] - theta[3] - theta[4])

Eq(nu[2], theta[1] + theta[2] - theta[3] - theta[4])

Eq(nu[3], -theta[1] - theta[2] + theta[3] + theta[4])

Eq(nu[4], -theta[1] - theta[2] + theta[3] + theta[4])

Eq(nu[2], nu[1])

Eq(nu[3], -nu[1])

Eq(nu[4], -nu[1])

and we see these $\nu$ relations are achievable from the modal overlap integrals (Note: unlike the $A_{eff}$ approximation, we do not need to assume $f_{i,j}=f_{i,j,k,l}$):

In [20]:
f_ijkl_integral = Eq(
    f[i,j,k,l], 
    Integral(conj(W(x,y,i))*conj(W(x,y,j))*W(x,y,k)*W(x,y,l),x,y)/
    sqrt(
        Integral(Abs(W(x,y,i))**2,x,y)*
        Integral(Abs(W(x,y,j))**2,x,y)*
        Integral(Abs(W(x,y,k))**2,x,y)*
        Integral(Abs(W(x,y,l))**2,x,y)
    )
)
f_ij_integral = Eq(
    f[i,j], 
    Integral(Abs(W(x,y,i))**2*Abs(W(x,y,j))**2,x,y)/
    (
        Integral(Abs(W(x,y,i))**2,x,y)*
        Integral(Abs(W(x,y,j))**2,x,y)
    )
)
f_ij_ii_jj = Eq(f[i,j], f[i,i,j,j])


f_ijkl_syms = [
    (f[2,1,3,4], f[1,2,3,4]),
    (f[3,4,1,2], conjugate(f[1,2,3,4])),
    (f[4,3,1,2], conjugate(f[1,2,3,4]))
]
f_ij_sym = Eq(f[j,i], f[i,j])


f_ij_integral
f_ijkl_integral
f_ij_ii_jj

f_ijkl_integral.subs([(i,1),(j,2),(k,3),(l,4)])
f_ijkl_integral.subs([(i,2),(j,1),(k,3),(l,4)])
f_ijkl_integral.subs([(i,3),(j,4),(k,1),(l,2)])
f_ijkl_integral.subs([(i,4),(j,3),(k,1),(l,2)])


for _args in f_ijkl_syms:
    Eq(*_args)
    
f_ij_sym

Eq(f[i, j], Integral(Abs(W(x, y, i))**2*Abs(W(x, y, j))**2, x, y)/(Integral(Abs(W(x, y, i))**2, x, y)*Integral(Abs(W(x, y, j))**2, x, y)))

Eq(f[i, j, k, l], Integral(W(x, y, k)*W(x, y, l)*conjugate(W(x, y, i))*conjugate(W(x, y, j)), x, y)/sqrt(Integral(Abs(W(x, y, i))**2, x, y)*Integral(Abs(W(x, y, j))**2, x, y)*Integral(Abs(W(x, y, k))**2, x, y)*Integral(Abs(W(x, y, l))**2, x, y)))

Eq(f[i, j], f[i, i, j, j])

Eq(f[1, 2, 3, 4], Integral(W(x, y, 3)*W(x, y, 4)*conjugate(W(x, y, 1))*conjugate(W(x, y, 2)), x, y)/sqrt(Integral(Abs(W(x, y, 1))**2, x, y)*Integral(Abs(W(x, y, 2))**2, x, y)*Integral(Abs(W(x, y, 3))**2, x, y)*Integral(Abs(W(x, y, 4))**2, x, y)))

Eq(f[2, 1, 3, 4], Integral(W(x, y, 3)*W(x, y, 4)*conjugate(W(x, y, 1))*conjugate(W(x, y, 2)), x, y)/sqrt(Integral(Abs(W(x, y, 1))**2, x, y)*Integral(Abs(W(x, y, 2))**2, x, y)*Integral(Abs(W(x, y, 3))**2, x, y)*Integral(Abs(W(x, y, 4))**2, x, y)))

...

Eq(f[4, 3, 1, 2], Integral(W(x, y, 1)*W(x, y, 2)*conjugate(W(x, y, 3))*conjugate(W(x, y, 4)), x, y)/sqrt(Integral(Abs(W(x, y, 1))**2, x, y)*Integral(Abs(W(x, y, 2))**2, x, y)*Integral(Abs(W(x, y, 3))**2, x, y)*Integral(Abs(W(x, y, 4))**2, x, y)))

Eq(f[2, 1, 3, 4], f[1, 2, 3, 4])

Eq(f[3, 4, 1, 2], conjugate(f[1, 2, 3, 4]))

Eq(f[4, 3, 1, 2], conjugate(f[1, 2, 3, 4]))

Eq(f[j, i], f[i, j])

In [21]:
simple_theta_choices = [
    (theta[1], nu[1]/4),
    (theta[2], nu[1]/4),
    (theta[3], -nu[1]/4),
    (theta[4], -nu[1]/4)
]
for _args in nu_subs:
    Eq(*_args).subs(simple_theta_choices)

True

Eq(nu[2], nu[1])

Eq(nu[3], -nu[1])

Eq(nu[4], -nu[1])

With these values for $\nu$ we obtain:

In [37]:
dBs = [
    bs(
        bs(
            _eq.subs(AB_subs + f_fc_subs), 
            lambda x: (x*exp(-I*theta[_i+1])).doit()
        ), 
        lambda x: x.subs(nu_subs).simplify().expand().collect(Abs(wave_mixing_fs[_i]), factor)
    )
    for _i, _eq in enumerate(dAs)
]
dBcs = [
    bs(
        bs(
            _eq.subs(AB_subs + f_fc_subs), 
            lambda x: (x*exp(I*theta[_i+1])).doit()
        ), 
        lambda x: x.subs(nu_subs).simplify().expand().collect(Abs(wave_mixing_fs[_i]), factor)
    )
    for _i, _eq in enumerate(dAcs)
]

for _eq in dBs:
    _eq
    
# for _eq in dBcs:
#     _eq

Eq(Derivative(B1(z), z), I*n2*(B1(z)*Bc1(z)*f[1, 1] + 2*B2(z)*Bc2(z)*f[1, 2] + 2*B3(z)*Bc3(z)*f[1, 3] + 2*B4(z)*Bc4(z)*f[1, 4])*B1(z)*omega[1]/c + 2*I*n2*B3(z)*B4(z)*Bc2(z)*exp(I*z*Delta(k))*Abs(f[1, 2, 3, 4])*omega[1]/c)

Eq(Derivative(B2(z), z), I*n2*(2*B1(z)*Bc1(z)*f[2, 1] + B2(z)*Bc2(z)*f[2, 2] + 2*B3(z)*Bc3(z)*f[2, 3] + 2*B4(z)*Bc4(z)*f[2, 4])*B2(z)*omega[2]/c + 2*I*n2*B3(z)*B4(z)*Bc1(z)*exp(I*z*Delta(k))*Abs(f[2, 1, 3, 4])*omega[2]/c)

Eq(Derivative(B3(z), z), I*n2*(2*B1(z)*Bc1(z)*f[3, 1] + 2*B2(z)*Bc2(z)*f[3, 2] + B3(z)*Bc3(z)*f[3, 3] + 2*B4(z)*Bc4(z)*f[3, 4])*B3(z)*omega[3]/c + 2*I*n2*B1(z)*B2(z)*Bc4(z)*exp(-I*z*Delta(k))*Abs(f[3, 4, 1, 2])*omega[3]/c)

Eq(Derivative(B4(z), z), I*n2*(2*B1(z)*Bc1(z)*f[4, 1] + 2*B2(z)*Bc2(z)*f[4, 2] + 2*B3(z)*Bc3(z)*f[4, 3] + B4(z)*Bc4(z)*f[4, 4])*B4(z)*omega[4]/c + 2*I*n2*B1(z)*B2(z)*Bc3(z)*exp(-I*z*Delta(k))*Abs(f[4, 3, 1, 2])*omega[4]/c)

We can then read off the required $a_{j}, a_{j,k}$, and define the $u(z,\mu_j), v(z,\mu_j)$ functions:

In [47]:
u_B_subs = [
    (u(z, mu[1]), r[1]*B1(z)*exp(I*beta[1]*z)), 
    (u(z, mu[2]), r[2]*B2(z)*exp(I*beta[2]*z)),
    (u(z, mu[3]), r[3]*Bc3(z)*exp(-I*beta[3]*z)),
    (u(z, mu[4]), r[4]*Bc4(z)*exp(-I*beta[4]*z)),
    (v(z, mu[1]), r[1]*Bc1(z)*exp(-I*beta[1]*z)), 
    (v(z, mu[2]), r[2]*Bc2(z)*exp(-I*beta[2]*z)),
    (v(z, mu[3]), r[3]*B3(z)*exp(I*beta[3]*z)),
    (v(z, mu[4]), r[4]*B4(z)*exp(I*beta[4]*z)),
]
sj_vals = [(s(1), 1), (s(2), 1), (s(3), -1), (s(4), -1)]
a_jk = Eq(a[j,k], -(2 - KroneckerDelta(j,k))*s(j)*I*n2*omega[j]/c*f[j,k]/r[k]**2)
a_jk_subs = [a_jk.subs([(j,_j),(k,_k)]).subs(sj_vals).args for _j in range(1,5) for _k in range(1,5)]
a_j_beta = Eq(a[j], -s(j)*I*beta[j])

a_j_subs = [a_j_beta.subs(j,_j).args for _j in range(1,5)]

rf_as = [
    Eq(Product(r[k],(k,1,4))/r[1]**2, 2*I*n2*omega[1]/c*Abs(f[1,2,3,4])),
    Eq(Product(r[k],(k,1,4))/r[2]**2, 2*I*n2*omega[2]/c*Abs(f[2,1,3,4])),
    Eq(Product(r[k],(k,1,4))/r[3]**2, -2*I*n2*omega[3]/c*Abs(f[3,4,1,2])),
    Eq(Product(r[k],(k,1,4))/r[4]**2, -2*I*n2*omega[4]/c*Abs(f[4,3,1,2]))
]

rk_prod_a = Eq(
    rf_as[0].lhs*rf_as[1].lhs*rf_as[2].lhs*rf_as[3].lhs, 
    rf_as[0].rhs*rf_as[1].rhs*rf_as[2].rhs*rf_as[3].rhs
).subs(r[1]*r[2]*r[3]*r[4], Product(r[k],(k,1,4)))

rk_prod = Eq(Product(r[k],(k,1,4)), solve(rk_prod_a, Product(r[k],(k,1,4)))[1])
 
rjs = [Eq(r[_j + 1]**2, solve(rf_as[_j].subs(*rk_prod.args), r[_j + 1]**2)[0]) for _j in range(4)]
rjs_subs = [_eq.args for _eq in rjs]

sqrt_prod_wv_mix = Abs(
    sqrt(wave_mixing_fs[0])*sqrt(wave_mixing_fs[1])*sqrt(wave_mixing_fs[2])*sqrt(wave_mixing_fs[3])
)

F_ks = [Eq(F[_k+1], Abs(wave_mixing_fs[_k])/sqrt_prod_wv_mix)  for _k in range(4)]

C_omega = Eq(C, sqrt(Product(omega[k], (k,1,4))))
rk_Fk = Eq(r[k]**2, -2*I*n2/c*s(k)/F[k]/(omega[k]/C_omega.rhs))
rks_Fk = [rk_Fk.subs(k,_k) for _k in range(1,5)]
a_jk_F = a_jk.subs([rk_Fk.args, (s(k), 1/s(k))])

_rk_checks = [
    Eq(
        rk_Fk.subs([C_omega.args, (k, _kk_+1), F_ks[_kk_].args]).doit()
       .subs(sj_vals).rhs.simplify(),
        rjs[_kk_].rhs
    ).simplify() for _kk_ in range(4)
]
for _ch in _rk_checks:
    if not _ch:
        raise ValueError('_rk_checks failed')
        
_ak_checks = [
    Eq(
        a_jk_F.subs([C_omega.args, (k, _kk_+1), F_ks[_kk_].args]).doit()
        .subs(sj_vals).simplify(),
        a_jk.subs(k, _kk_+1).subs([rjs[_kk_].args]).simplify()
    ).simplify() for _kk_ in range(4)
]
for _ch in _ak_checks:
    if not _ch:
        raise ValueError('_ak_checks failed')
        
a_j_beta
a_jk

for _eq in rf_as:
    _eq.doit()

for _args in u_B_subs:
    Eq(*_args)

rk_prod_a
rk_prod

for _eq in rjs:
    _eq

for _eq in F_ks:
    _eq

C_omega
a_jk_F

Eq(a[j], -I*s(j)*beta[j])

Eq(a[j, k], I*n2*(KroneckerDelta(j, k) - 2)*s(j)*f[j, k]*omega[j]/(r[k]**2*c))

Eq(r[2]*r[3]*r[4]/r[1], 2*I*n2*Abs(f[1, 2, 3, 4])*omega[1]/c)

Eq(r[1]*r[3]*r[4]/r[2], 2*I*n2*Abs(f[2, 1, 3, 4])*omega[2]/c)

Eq(r[1]*r[2]*r[4]/r[3], -2*I*n2*Abs(f[3, 4, 1, 2])*omega[3]/c)

...

Eq(F[2], Abs(f[2, 1, 3, 4])/Abs(sqrt(f[1, 2, 3, 4])*sqrt(f[2, 1, 3, 4])*sqrt(f[3, 4, 1, 2])*sqrt(f[4, 3, 1, 2])))

Eq(F[3], Abs(f[3, 4, 1, 2])/Abs(sqrt(f[1, 2, 3, 4])*sqrt(f[2, 1, 3, 4])*sqrt(f[3, 4, 1, 2])*sqrt(f[4, 3, 1, 2])))

Eq(F[4], Abs(f[4, 3, 1, 2])/Abs(sqrt(f[1, 2, 3, 4])*sqrt(f[2, 1, 3, 4])*sqrt(f[3, 4, 1, 2])*sqrt(f[4, 3, 1, 2])))

Eq(C, sqrt(Product(omega[k], (k, 1, 4))))

Eq(a[j, k], -(KroneckerDelta(j, k) - 2)*s(j)*s(k)*F[k]*f[j, k]*omega[j]*omega[k]/(2*sqrt(Product(omega[k], (k, 1, 4)))))

In [48]:
_dB_uvs = [duj.subs(j,1), duj.subs(j,2), dvj.subs(j,3), dvj.subs(j,4)]
_dBc_uvs = [dvj.subs(j,1), dvj.subs(j,2), duj.subs(j,3), duj.subs(j,4)]
Bjzs = [B1(z), B2(z), B3(z), B4(z)]
Bcjzs = [Bc1(z), Bc2(z), Bc3(z), Bc4(z)]
f_abs = [
    Abs(f[1,2,3,4]),
    Abs(f[2,1,3,4]),
    Abs(f[3,4,1,2]),
    Abs(f[4,3,1,2])
]

dBs_from_u_v = [
    Eq(
        (_dB_uvs[_j].doit().subs(u_B_subs).lhs*exp(-I*beta[_j+1]*z)/r[_j+1])
        .subs(sj_vals).doit().simplify() 
        - Bjzs[_j]*I*beta[_j+1], 
        (
            (_dB_uvs[_j].doit().subs(u_B_subs).rhs*exp(-I*beta[_j+1]*z)/r[_j+1])
            .expand().subs(*rf_as[_j].doit().args)
            .subs(a_jk_subs)
            .subs(sj_vals)
            .subs(a_j_subs)
            - Bjzs[_j]*I*beta[_j+1]
        ).subs(sj_vals).collect(f_abs, simplify).subs(beta_4_delta_k_sub)
    )
    for _j in range(4)
]
dBcs_from_u_v = [
    Eq(
        (_dBc_uvs[_j].doit().subs(u_B_subs).lhs*exp(I*beta[_j+1]*z)/r[_j+1])
        .subs(sj_vals).doit().simplify()
         + Bcjzs[_j]*I*beta[_j+1], 
        (
            (_dBc_uvs[_j].doit().subs(u_B_subs).rhs*exp(I*beta[_j+1]*z)/r[_j+1])
            .expand().subs(*rf_as[_j].doit().args)
            .subs(a_jk_subs)
            .subs(sj_vals)
            .subs(a_j_subs)
            + Bcjzs[_j]*I*beta[_j+1]
        ).subs(sj_vals).collect(f_abs, simplify).subs(beta_4_delta_k_sub)
    )
    for _j in range(4)
]

im_theta_parts = [(im(theta[_j]), 0) for _j in range(1,5)]
dBs_from_u_v_checks = [
    not Eq(
        (dBs[_j].lhs - dBs_from_u_v[_j].lhs).expand(), 
        (dBs[_j].rhs - dBs_from_u_v[_j].rhs).expand()
    )
    for _j in range(4)
]
dBcs_from_u_v_checks = [
    not Eq(
        (dBcs[_j].lhs - dBcs_from_u_v[_j].lhs).expand(), 
        (dBcs[_j].rhs - dBcs_from_u_v[_j].rhs).subs(im_theta_parts).expand()
    )
    for _j in range(4)
]
if any(dBs_from_u_v_checks):
    raise ValueError('dBs_from_u_v_checks failed')
if any(dBcs_from_u_v_checks):
    raise ValueError('dBcs_from_u_v_checks failed')
    

# for _eq in dBs_from_u_v:
#     _eq
    
# for _eq in dBcs_from_u_v:
#     _eq

We note from the earlier overlap integrals that in fact $F_{k}$ reduces to a single constant leading to a symmetric matrix for $a_{j,k}$, and simple scale factors relating $A$ to $u,v$:

In [40]:
BA_subs = [
    (B1(z), A1(z)*exp(-I*theta[1])),
    (B2(z), A2(z)*exp(-I*theta[2])),
    (B3(z), A3(z)*exp(-I*theta[3])),
    (B4(z), A4(z)*exp(-I*theta[4])),
    (Bc1(z), Ac1(z)*exp(I*theta[1])),
    (Bc2(z), Ac2(z)*exp(I*theta[2])),
    (Bc3(z), Ac3(z)*exp(I*theta[3])),
    (Bc4(z), Ac4(z)*exp(I*theta[4]))
]

F_k_simple = Eq(F[k], 1/Abs(f[1,2,3,4]))
Fk_checks = [
    (not _eq.subs(f_ijkl_syms)
     .simplify().subs(*F_k_simple.subs(k,_k+1).args)
    )
    for _k, _eq in enumerate(F_ks)
]
if any(Fk_checks):
    raise ValueError('Fk_checks failed')
    
a_jk_F_sym = a_jk_F.subs([F_k_simple.args])
a_jk_F_sym_subs = [a_jk_F_sym.subs([(j, _j+1),(k, _k+1)]).args for _j in range(4) for _k in range(4)]

T_C_f_ijkl = Eq(T, sqrt(2)*sqrt(C)*sqrt(n2)*sqrt(Abs(f[1,2,3,4]))/sqrt(c))
r_k_T= Eq(r[k], exp(-s(k)*I*pi/4)*T/sqrt(omega[k]))

nu_f = Eq(exp(I*nu[1]), solve(Eq(*f_fc_subs[0]),exp(I*nu[1]))[0])

u_A_subs = [
    Eq(*u_B_subs[_j]).subs(BA_subs)
    .subs([r_k_T.subs(k,(_j % 4) + 1).args])
    .subs(simple_theta_choices + sj_vals)
    .args
    for _j in range(8)
]


F_k_simple
a_jk_F_sym
T_C_f_ijkl
nu_f

for _args in u_A_subs:
    Eq(*_args)

Eq(F[k], 1/Abs(f[1, 2, 3, 4]))

Eq(a[j, k], -(KroneckerDelta(j, k) - 2)*s(j)*s(k)*f[j, k]*omega[j]*omega[k]/(2*Abs(f[1, 2, 3, 4])*sqrt(Product(omega[k], (k, 1, 4)))))

Eq(T, sqrt(2)*sqrt(C)*sqrt(n2)*sqrt(Abs(f[1, 2, 3, 4]))/sqrt(c))

Eq(exp(I*nu[1]), Abs(f[1, 2, 3, 4])/conjugate(f[1, 2, 3, 4]))

Eq(u(z, mu[1]), T*A1(z)*exp(-I*pi/4)*exp(-I*nu[1]/4)*exp(I*z*beta[1])/sqrt(omega[1]))

...

Eq(u(z, mu[4]), T*Ac4(z)*exp(I*pi/4)*exp(-I*nu[1]/4)*exp(-I*z*beta[4])/sqrt(omega[4]))

Eq(v(z, mu[1]), T*Ac1(z)*exp(-I*pi/4)*exp(I*nu[1]/4)*exp(-I*z*beta[1])/sqrt(omega[1]))

Eq(v(z, mu[2]), T*Ac2(z)*exp(-I*pi/4)*exp(I*nu[1]/4)*exp(-I*z*beta[2])/sqrt(omega[2]))

Eq(v(z, mu[3]), T*A3(z)*exp(I*pi/4)*exp(I*nu[1]/4)*exp(I*z*beta[3])/sqrt(omega[3]))

Eq(v(z, mu[4]), T*A4(z)*exp(I*pi/4)*exp(I*nu[1]/4)*exp(I*z*beta[4])/sqrt(omega[4]))

In [41]:
u_A_order_map = [
    _x[1].subs([
        (T,1), 
        (nu[1],0), 
        (pi,0), 
        (beta[1],0), (beta[2],0), (beta[3],0), (beta[4],0),
        (omega[1], 1), (omega[2], 1), (omega[3], 1), (omega[4], 1)
    ]) 
    for _x in u_A_subs
]

dAs_from_u_v = [
    Eq(
        diff(u_A_order_map[_j],z), 
        solve(
            (duj if _j < 4 else dvj).subs(j, (_j % 4) + 1).doit()
            .subs(u_A_subs).doit()
            .subs(a_jk_F_sym_subs)
            .subs(a_j_subs)
            .subs(sj_vals)
            .subs([T_C_f_ijkl.args, C_omega.args])
            , 
            diff(u_A_order_map[_j],z)
        )[0]
        .subs(delta_k.rhs,  delta_k.lhs)
        .expand()
        .subs([nu_f.args])
        .subs(
            sqrt(omega[1]*omega[2]*omega[3]*omega[4]), 
            sqrt(omega[1])*sqrt(omega[2])*sqrt(omega[3])*sqrt(omega[4])
        )
        .subs(beta[4], solve(delta_k, beta[4])[0])
        .simplify()
        .expand()
    )
    for _j in range(8)
]

dA_and_dAcs = dAs + dAcs
dAs_from_u_v_rearranged = [
    dAs_from_u_v[0], 
    dAs_from_u_v[1],
    dAs_from_u_v[6],
    dAs_from_u_v[7],
    dAs_from_u_v[4], 
    dAs_from_u_v[5],
    dAs_from_u_v[2],
    dAs_from_u_v[3],
]
dAs_from_du_dv_checks = [
    (
        (dA_and_dAcs[_k].rhs - dAs_from_u_v_rearranged[_k].rhs)
        .expand().subs(f_ijkl_syms).simplify()
        .subs(Abs(f[1,2,3,4])**2, f[1,2,3,4]*conjugate(f[1,2,3,4]))
    ) != 0 
    for _k in range(8)
]
if any(dAs_from_du_dv_checks):
    raise ValueError('dAs_from_du_dv_checks failed')

for _eq in dAs_from_u_v_rearranged[0:4]:
    _eq

Eq(Derivative(A1(z), z), I*n2*A1(z)**2*Ac1(z)*f[1, 1]*omega[1]/c + 2*I*n2*A1(z)*A2(z)*Ac2(z)*f[1, 2]*omega[1]/c + 2*I*n2*A1(z)*A3(z)*Ac3(z)*f[1, 3]*omega[1]/c + 2*I*n2*A1(z)*A4(z)*Ac4(z)*f[1, 4]*omega[1]/c + 2*I*n2*A3(z)*A4(z)*Ac2(z)*exp(I*z*Delta(k))*Abs(f[1, 2, 3, 4])**2*omega[1]/(conjugate(f[1, 2, 3, 4])*c))

Eq(Derivative(A2(z), z), 2*I*n2*A1(z)*A2(z)*Ac1(z)*f[2, 1]*omega[2]/c + I*n2*A2(z)**2*Ac2(z)*f[2, 2]*omega[2]/c + 2*I*n2*A2(z)*A3(z)*Ac3(z)*f[2, 3]*omega[2]/c + 2*I*n2*A2(z)*A4(z)*Ac4(z)*f[2, 4]*omega[2]/c + 2*I*n2*A3(z)*A4(z)*Ac1(z)*exp(I*z*Delta(k))*Abs(f[1, 2, 3, 4])**2*omega[2]/(conjugate(f[1, 2, 3, 4])*c))

Eq(Derivative(A3(z), z), 2*I*n2*A1(z)*A2(z)*Ac4(z)*exp(-I*z*Delta(k))*conjugate(f[1, 2, 3, 4])*omega[3]/c + 2*I*n2*A1(z)*A3(z)*Ac1(z)*f[3, 1]*omega[3]/c + 2*I*n2*A2(z)*A3(z)*Ac2(z)*f[3, 2]*omega[3]/c + I*n2*A3(z)**2*Ac3(z)*f[3, 3]*omega[3]/c + 2*I*n2*A3(z)*A4(z)*Ac4(z)*f[3, 4]*omega[3]/c)

Eq(Derivative(A4(z), z), 2*I*n2*A1(z)*A2(z)*Ac3(z)*exp(-I*z*Delta(k))*conjugate(f[1, 2, 3, 4])*omega[4]/c + 2*I*n2*A1(z)*A4(z)*Ac1(z)*f[4, 1]*omega[4]/c + 2*I*n2*A2(z)*A4(z)*Ac2(z)*f[4, 2]*omega[4]/c + 2*I*n2*A3(z)*A4(z)*Ac3(z)*f[4, 3]*omega[4]/c + I*n2*A4(z)**2*Ac4(z)*f[4, 4]*omega[4]/c)

## Finding phase modulation polynomial Q from the equations of motion

In [42]:
duj
dvj

Eq(Derivative(u(z, mu[j]), z), -(a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*u(z, mu[j]) + Product(v(z, mu[k]), (k, 1, 4))/v(z, mu[j]))

Eq(Derivative(v(z, mu[j]), z), (a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*v(z, mu[j]) - Product(u(z, mu[k]), (k, 1, 4))/u(z, mu[j]))

In [59]:
dQ_uj = Eq(
    Derivative(Q(u(z,mu[1])*v(z,mu[1]), u(z,mu[n])*v(z,mu[n])),v(z, mu[j])), 
    du_phase_mod_part
)
dQ_vj = Eq(
    Derivative(Q(u(z,mu[1])*v(z,mu[1]), u(z,mu[n])*v(z,mu[n])),u(z, mu[j])),
    -dv_phase_mod_part.factor()
)

Q_poly  = Eq(
    Q(psi[1], psi[2], psi[3], psi[4]), 
    a[0] + Sum(a[j]*psi[j], (j,1,4)) + Sum(a[j,k]/2*psi[j]*psi[k], (j,1,4), (k,1,4))
)

ajk_syms = [(a[j,k], a[k,j]) for j in range(1,5) for k in range(1,5) if j > k]

# for _args in ajk_syms:
#     Eq(*_args)

psi_uv_subs = [
    (psi[1], u(z,mu[1])*v(z,mu[1])),
    (psi[2], u(z,mu[2])*v(z,mu[2])),
    (psi[3], u(z,mu[3])*v(z,mu[3])),
    (psi[4], u(z,mu[4])*v(z,mu[4]))
]
dQ_ujs = [
    bs(
        Q_poly.doit().subs(psi_uv_subs).expand(), 
        lambda x: diff(x, v(z,mu[_j])).factor()
    ).subs(ajk_syms).simplify()
    for _j in range(1,5)
]
dQ_vjs = [
    bs(
        Q_poly.doit().subs(psi_uv_subs).expand(), 
        lambda x: diff(x, u(z,mu[_j])).factor()
    ).subs(ajk_syms).simplify()
    for _j in range(1,5)
]


dQ_uj
dQ_vj

Q_poly

for _j in range(4):
    (dQ_ujs[_j].rhs.subs(ajk_syms) - dQ_uj.rhs.subs(j,_j+1).doit().subs(ajk_syms)) == 0
    (dQ_vjs[_j].rhs.subs(ajk_syms) - dQ_vj.rhs.subs(j,_j+1).doit().subs(ajk_syms)) == 0

Eq(Derivative(Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[n])*v(z, mu[n])), v(z, mu[j])), (a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*u(z, mu[j]))

Eq(Derivative(Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[n])*v(z, mu[n])), u(z, mu[j])), (a[j] + Sum(u(z, mu[k])*v(z, mu[k])*a[j, k], (k, 1, 4)))*v(z, mu[j]))

Eq(Q(psi[1], psi[2], psi[3], psi[4]), a[0] + Sum(a[j]*psi[j], (j, 1, 4)) + Sum(a[j, k]*psi[j]*psi[k]/2, (j, 1, 4), (k, 1, 4)))

True

True

...

True

True

True

True

True

## Solving for the generating function

Following the unified theory, we introduce $\rho(z)$ and the offsets $\gamma_j$ and define the modal powers using them. We further need to impose a constraint on $\gamma_j$ as the theory is invariant under $\rho \rightarrow  \rho + c$, $\gamma_j \rightarrow  \gamma_j + c$ which prevents us from determining all $\gamma_j$. We will choose the zero-mean normalisation such that:

In [44]:
uv_j_rho = Eq(u(z,mu[j])*v(z,mu[j]), gamma[j] - rho(z))
sum_gamma_j_0 = Eq(Sum(gamma[j], (j,1,4)), 0)
rho_z_mean_uv = Eq(rho(z), -Sum(u(z,mu[j])*v(z,mu[j]), (j,1,4))/4)
rho_z_mean_uv_init = rho_z_mean_uv.subs(z,0)
gamma_j_init = Eq(gamma[j], solve(uv_j_rho, gamma[j])[0].subs(*rho_z_mean_uv.args).subs(z,0))
drho_uv = Eq(
    -(duj.lhs*v(z, mu[j]) + dvj.lhs*u(z, mu[j]))
    .subs(diff(uv_j_rho.lhs,z), diff(uv_j_rho.rhs,z)), 
    -(duj.rhs*v(z, mu[j]) + dvj.rhs*u(z, mu[j])).simplify()
)
drho_uv_init = drho_uv.subs(z,0)

uv_j_rho
sum_gamma_j_0
rho_z_mean_uv
rho_z_mean_uv_init
gamma_j_init
drho_uv
drho_uv_init

Eq(u(z, mu[j])*v(z, mu[j]), -rho(z) + gamma[j])

Eq(Sum(gamma[j], (j, 1, 4)), 0)

Eq(rho(z), -Sum(u(z, mu[j])*v(z, mu[j]), (j, 1, 4))/4)

Eq(rho(0), -Sum(u(0, mu[j])*v(0, mu[j]), (j, 1, 4))/4)

Eq(gamma[j], u(0, mu[j])*v(0, mu[j]) - Sum(u(0, mu[j])*v(0, mu[j]), (j, 1, 4))/4)

Eq(Derivative(rho(z), z), u(z, mu[1])*u(z, mu[2])*u(z, mu[3])*u(z, mu[4]) - v(z, mu[1])*v(z, mu[2])*v(z, mu[3])*v(z, mu[4]))

Eq(Subs(Derivative(rho(z), z), z, 0), u(0, mu[1])*u(0, mu[2])*u(0, mu[3])*u(0, mu[4]) - v(0, mu[1])*v(0, mu[2])*v(0, mu[3])*v(0, mu[4]))

In [45]:
drhop = Eq(diff(rho(z),z)**2, Q(gamma[1] - rho(z), gamma[N] - rho(z))**2 - 4*Product(gamma[j] - rho(z),(j,1,N)))
drhopp = Eq(diff(rho(z),z), rhop(z))


drhop
drhopp

Eq(Derivative(rho(z), z)**2, Q(-rho(z) + gamma[1], -rho(z) + gamma[N])**2 - 4*Product(-rho(z) + gamma[j], (j, 1, N)))

Eq(Derivative(rho(z), z), \\rho'(z))

### Expressing $Q$ in terms of $\rho(z)$

First, we arrange $\rho(z)$ as a hyperelliptic differential equation and perform the book keeping for various intermediate coefficients. We remark that the $b_j$ coefficients are just the Taylor series coefficients of $Q(\rho)$ expanded about $\rho=0$.

In [32]:
a_0_eq = Eq(
    a[0],
    - Sum(a[j]*u(z,mu[j])*v(z,mu[j]), (j,1,4))
    - Sum(a[j,k]/2*u(z,mu[j])*v(z,mu[j])*u(z,mu[k])*v(z,mu[k]), (j,1,4), (k,1,4))
    + Product(u(z,mu[j]),(j,1,4)) + Product(v(z,mu[j]),(j,1,4))
)
a_0_init = a_0_eq.subs(z,0)

Q_poly_uv  = Eq(
    Q(u(z,mu[1])*v(z,mu[1]), u(z,mu[2])*v(z,mu[2]), u(z,mu[3])*v(z,mu[3]), u(z,mu[4])*v(z,mu[4])), 
    a[0] + Sum(a[j]*(gamma[j] - rho(z)), (j,1,4)) + 
    Sum(a[j,k]/2*(gamma[j] - rho(z))*(gamma[k] - rho(z)), (j,1,4), (k,1,4))
)
Q_poly_uv_b  = Eq(
    Q(u(z,mu[1])*v(z,mu[1]), u(z,mu[2])*v(z,mu[2]), u(z,mu[3])*v(z,mu[3]), u(z,mu[4])*v(z,mu[4])), 
    b[0] + rho(z)*b[1] + rho(z)**2*b[2]
)

# Depend on a[j], a[j,k], gamma[j]
b_j_coeffs = [
    Eq(b[0], a[0] + Sum(a[j]*gamma[j], (j,1,4)) + Sum(a[j,k]/2*gamma[j]*gamma[k], (j,1,4), (k,1,4))),
    Eq(b[1], -Sum(a[j,k]/2*(gamma[j] + gamma[k]), (j,1,4), (k,1,4)) - Sum(a[j], (j,1,4))),
    Eq(b[2], Sum(a[j,k]/2, (j,1,4), (k,1,4)))
]

# Depend on gamma[j]
c_j_coeffs = [
    Eq(c[0], Product(gamma[j], (j,1,4))),
    Eq(c[1], -Sum(Product(gamma[j], (j, 1, k-1))*Product(gamma[j], (j, k+1, 4)), (k, 1, 4))),
    Eq(c[2], -Sum(((gamma[j] - gamma[k])**2)/4,(j,1,4), (k,1,4)) + 3*Sum((gamma[j]**2)/2, (j,1,4))),
    Eq(c[3], -Sum(gamma[j], (j,1,4))).subs([sum_gamma_j_0.args]),
    Eq(c[4], 1)
]

drhop_b = Eq(diff(rho(z),z)**2, (Q_poly_uv_b.rhs)**2 - 4*Product(gamma[j] - rho(z),(j,1,4)))

rho_c_prod_to_sum = Eq(Product(-rho(z) + gamma[j], (j, 1, 4)), Sum(c[j]*rho(z)**j, (j,0,4)))
drhop_c = Eq(drhop_b.lhs, drhop_b.rhs.subs(*rho_c_prod_to_sum.args).expand().collect(rho(z), factor))

d_j_coeffs = [Eq(d[_j], drhop_c.doit().rhs.coeff(rho(z),_j)) for _j in range(5)]
drhop_d = Eq(drhop_b.lhs, Sum(d[j]*rho(z)**j, (j,0,4)))

_rhop_bcd_check = (
    (drhop_b.rhs.doit() - drhop_d.rhs.doit())
    .subs([_eq.args for _eq in d_j_coeffs])
    .subs([_eq.args for _eq in c_j_coeffs])
    .doit().expand().factor().subs(*sum_gamma_j_0.doit().args)
)
if not _rhop_bcd_check == 0:
    raise ValueError('_rhop_bcd_check failed')

a_0_eq
a_0_init
Q_poly_uv
Q_poly_uv_b

for _eq in b_j_coeffs:
    _eq

for _eq in c_j_coeffs:
    _eq

for _eq in d_j_coeffs:
    _eq

rho_c_prod_to_sum

drhop_b
drhop_d

Eq(a[0], Product(u(z, mu[j]), (j, 1, 4)) + Product(v(z, mu[j]), (j, 1, 4)) - Sum(u(z, mu[j])*v(z, mu[j])*a[j], (j, 1, 4)) - Sum(u(z, mu[j])*u(z, mu[k])*v(z, mu[j])*v(z, mu[k])*a[j, k]/2, (j, 1, 4), (k, 1, 4)))

Eq(a[0], Product(u(0, mu[j]), (j, 1, 4)) + Product(v(0, mu[j]), (j, 1, 4)) - Sum(u(0, mu[j])*v(0, mu[j])*a[j], (j, 1, 4)) - Sum(u(0, mu[j])*u(0, mu[k])*v(0, mu[j])*v(0, mu[k])*a[j, k]/2, (j, 1, 4), (k, 1, 4)))

Eq(Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[2])*v(z, mu[2]), u(z, mu[3])*v(z, mu[3]), u(z, mu[4])*v(z, mu[4])), a[0] + Sum((-rho(z) + gamma[j])*a[j], (j, 1, 4)) + Sum((-rho(z) + gamma[j])*(-rho(z) + gamma[k])*a[j, k]/2, (j, 1, 4), (k, 1, 4)))

Eq(Q(u(z, mu[1])*v(z, mu[1]), u(z, mu[2])*v(z, mu[2]), u(z, mu[3])*v(z, mu[3]), u(z, mu[4])*v(z, mu[4])), rho(z)**2*b[2] + rho(z)*b[1] + b[0])

Eq(b[0], a[0] + Sum(a[j]*gamma[j], (j, 1, 4)) + Sum(a[j, k]*gamma[j]*gamma[k]/2, (j, 1, 4), (k, 1, 4)))

...

Eq(d[3], 2*b[1]*b[2] - 4*c[3])

Eq(d[4], b[2]**2 - 4*c[4])

Eq(Product(-rho(z) + gamma[j], (j, 1, 4)), Sum(rho(z)**j*c[j], (j, 0, 4)))

Eq(Derivative(rho(z), z)**2, (rho(z)**2*b[2] + rho(z)*b[1] + b[0])**2 - 4*Product(-rho(z) + gamma[j], (j, 1, 4)))

Eq(Derivative(rho(z), z)**2, Sum(rho(z)**j*d[j], (j, 0, 4)))

We then solve for $\rho(z)$ in terms of the Weierstrass $\wp$ elliptic function:

In [33]:
rho_lam = Eq(diff(rho(z),z)**2, d[4]*Product(rho(z) - lam[j],(j,1,4)))
rho_q_lam1 = Eq(rho(z), q(z) + lam[1])
qz_lam = rho_lam.subs(*rho_q_lam1.args).doit()


# -- q to s transform --

q_s = Eq(q(z), 1/s(z))
sz_lam = qz_lam.subs(*q_s.args).doit()
Omega_lams = [Eq(Omega[_j], -1/(lam[1] - lam[_j+1])) for _j in range(1,4)]
sz_lam_b = Eq(
    Derivative(s(z), z)**2,
    -(lam[1] - lam[2])*(lam[1] - lam[3])*(lam[1] - lam[4])*
    (s(z) - 1/(lam[1] - lam[2]))*(s(z) - 1/(lam[1] - lam[3]))*(s(z) - 1/(lam[1] - lam[4]))*d[4]
)
sz_lam_c = Eq(
    Derivative(s(z), z)**2, 
    -Product(s(z) - Omega[j],(j,1,3))*d[4]/Product(Omega[j],(j,1,3))
)

_q_to_s_test = rho_lam.subs([
    rho_q_lam1.args,
    qz_lam.args,
    q_s.args
]).doit()
_q_to_s_test = Eq(_q_to_s_test.lhs*s(z)**4, _q_to_s_test.rhs*s(z)**4)
_q_to_s_test_lhs = _q_to_s_test.lhs - sz_lam_c.lhs
_q_to_s_test_rhs = (
    _q_to_s_test.rhs - sz_lam_c.rhs.doit().subs([_eq.args for _eq in Omega_lams])
).expand().simplify()
if not _q_to_s_test_lhs == 0:
    raise ValueError('_q_to_s_test_lhs failed')
if not _q_to_s_test_rhs == 0:
    raise ValueError('_q_to_s_test_lhs failed')
    
    
# -- s to w transform -- 

sz_w_Omega = Eq(s(z), -4*Product(Omega[l],(l,1,3))/d[4]*w(z) + Sum(Omega[l]/3,(l,1,3)))

g2_Omega = Eq(
    g2, 
    d[4]**2*Sum((Omega[j] - Omega[k])**2, (j,1,3), (k,1,3))/48/Product(Omega[j],(j,1,3))**2
)
g3_Omega = Eq(
    g3, 
    -(Product(Omega[j] - Sum(Omega[k]/3,(k,1,3)),(j,1,3))*d[4]**3)/16/Product(Omega[j],(j,1,3))**3
)

dw_g2_g3 = Eq(diff(w(z),z)**2, 4*w(z)**3 - g2*w(z) - g3)
wz_wp_sol = Eq(w(z), wp(z-z0, g2, g3))

_sw_check = (
    sz_lam_c
    .subs(*sz_w_Omega.args).doit()
    .subs(*dw_g2_g3.subs([g2_Omega.args, g3_Omega.args]).args)
    .simplify()
)
if not _sw_check:
    raise ValueError('_sw_check failed')
    
    
## -- define z1 -- 

rho_wp = rho_q_lam1.subs([q_s.args, sz_w_Omega.args, wz_wp_sol.args])

dw_xi = Eq(diff(w(z),z)**2, 4*Product(w(z) - xi[j], (j,1,3)))
xi_j_omega = Eq(xi[j], (Sum(Omega[l]/3, (l, 1, 3)) - Omega[j])/(4*Product(Omega[l], (l, 1, 3))/d[4]))
xi_j_omega_sum = Eq(Sum(xi[j], (j,1,3)), Sum(xi[j].subs(*xi_j_omega.args), (j,1,3)).doit().simplify())
xi_j_omega_sum_b = Eq(xi[j]*(4*Product(Omega[l], (l, 1, 3)))/d[4] + Omega[j], Sum(Omega[l]/3, (l, 1, 3)))
xi_j_root = Eq(xi[j], wp(omega[j], g2, g3))

wp_z1 = Eq(wp(z1, g2, g3), Sum(Omega[l]/3, (l, 1, 3))/(4*Product(Omega[l], (l, 1, 3))/d[4]))
wpp_z1 = Eq(wpp(z1, g2, g3), d[4]*sqrt(d[4])/(4*Product(Omega[l], (l, 1, 3))))

wpp_z1_sqrd = Eq(
    wpp(z1,g2,g3)**2, 
    (
        (4*wp(z1, g2, g3)**3 - g2*wp(z1, g2, g3) - g3).subs([
            wp_z1.args,
            g2_Omega.args,
            g3_Omega.args
        ]).expand().doit().expand()
        - d[4]**3/16/Product(Omega[j],(j,1,3))**2 
    ).simplify()
    + d[4]**3/16/Product(Omega[j],(j,1,3))**2
).subs([(Product(Omega[j], (j, 1, 3)), Product(Omega[l], (l, 1, 3)))])

z1_wpp_sqrd_check = Eq(wpp_z1.lhs**2 - wpp_z1_sqrd.lhs, wpp_z1.rhs**2 - wpp_z1_sqrd.rhs)
if not z1_wpp_sqrd_check:
    raise ValueError('z1_wpp_sqrd_check failed')
    
## -- 
    
rho_z_sqrt_d4 = Eq(
    rho(z),
    lam[1] + 
    wpp_z1.lhs/wpp_z1.rhs/(
        -wp(z - z0, g2, g3) + 
        Sum(Omega[l]/3, (l, 1, 3))/(4*Product(Omega[l], (l, 1, 3))/d[4])
    ).subs(wp_z1.rhs, wp_z1.lhs)/(4*Product(Omega[l], (l, 1, 3))/d[4])
)

rho_z_sqrt_d4_test = Eq(
    rho_z_sqrt_d4.subs([wp_z1.args]).lhs,
    (rho_z_sqrt_d4.subs([wp_z1.args]).rhs - lam[1])*wpp_z1.rhs/wpp_z1.lhs + lam[1]
)
rho_z_sqrt_d4_check_lhs = (rho_wp.rhs - rho_z_sqrt_d4_test.rhs).simplify() == 0
rho_z_sqrt_d4_check_rhs = (rho_wp.rhs - rho_z_sqrt_d4_test.rhs).simplify() == 0
if not rho_z_sqrt_d4_check_lhs:
    raise ValueError('rho_z_sqrt_d4_check_lhs failed')
if not rho_z_sqrt_d4_check_rhs:
    raise ValueError('rho_z_sqrt_d4_check_rhs failed')


## -- zeta function expansion --

zeta_z_z0_z1 = pw_to_zw_identity_one_sided.subs([(z,t),(x,y)]).subs([(t, z1), (y, z - z0)])
rho_zeta = Eq(
    rho_z_sqrt_d4.lhs, 
    (rho_z_sqrt_d4.rhs - lam[1] + zeta_z_z0_z1.lhs/sqrt(d[4])).simplify() + 
    lam[1] - zeta_z_z0_z1.rhs/sqrt(d[4])
)

## -- Derivative of rho as wp -- 

drho_zeta_d4 = Eq(diff(rho_zeta.lhs,z), diff(rho_zeta.rhs,z).expand())
drho_wp_d4 = Eq(
    Derivative(rho(z), z),
    -(wp(z - z0 + z1, g2, g3) - wp(z - z0 - z1,g2,g3))/sqrt(d[4])
)
drho_wp_d4_b = Eq(
    diff(rho_z_sqrt_d4.lhs, z), 
    diff(rho_z_sqrt_d4.rhs, z).subs(diff(wp(z-z0,g2,g3),z), wpp(z-z0,g2,g3))
)
# Note this is true for all z,z1
drho_wp_id = Eq(drho_wp_d4.lhs/drho_wp_d4_b.lhs, drho_wp_d4.rhs/drho_wp_d4_b.rhs).subs(z,z+z0)

rho_lam
rho_q_lam1
qz_lam
q_s

for _eq in Omega_lams:
    _eq

sz_lam_c

sz_w_Omega
g2_Omega
g3_Omega
dw_g2_g3
wz_wp_sol
rho_wp
xi_j_omega
xi_j_omega_sum
xi_j_omega_sum_b
xi_j_root

wp_z1
wpp_z1
rho_z_sqrt_d4
rho_zeta

drho_wp_d4
drho_wp_d4_b
drho_wp_id

Eq(Derivative(rho(z), z)**2, d[4]*Product(rho(z) - lambda[j], (j, 1, 4)))

Eq(rho(z), q(z) + lambda[1])

Eq(Derivative(q(z), z)**2, (q(z) + lambda[1] - lambda[2])*(q(z) + lambda[1] - lambda[3])*(q(z) + lambda[1] - lambda[4])*q(z)*d[4])

Eq(q(z), 1/s(z))

Eq(Omega[1], -1/(lambda[1] - lambda[2]))

...

Eq(rho(z), lambda[1] + \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(d[4])))

Eq(rho(z), -(2*zeta(z1, g2, g3) - zeta(-z + z0 + z1, g2, g3) - zeta(z - z0 + z1, g2, g3))/sqrt(d[4]) + lambda[1])

Eq(Derivative(rho(z), z), (wp(z - z0 - z1, g2, g3) - wp(z - z0 + z1, g2, g3))/sqrt(d[4]))

Eq(Derivative(rho(z), z), \wp\'(z1, g2, g3)*\wp\'(z - z0, g2, g3)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))**2*sqrt(d[4])))

Eq(1, (-wp(z, g2, g3) + wp(z1, g2, g3))**2*(wp(z - z1, g2, g3) - wp(z + z1, g2, g3))/(\wp\'(z, g2, g3)*\wp\'(z1, g2, g3)))

In [34]:
# Useful if need to take limit where d[4] -> 0
rho_z_no_sqrt_d4 = rho_z_sqrt_d4.subs([
    (sqrt(d[4]), solve(rho_z_sqrt_d4.subs(z,0), sqrt(d[4]))[0]),
    (wp(-z0,g2,g3), wp(z0,g2,g3))
]).simplify()
rho_z_no_sqrt_d4 = Eq(
    rho_z_no_sqrt_d4.lhs,
    apart(rho_z_no_sqrt_d4.rhs.subs(wp(z-z0,g2,g3), X),X).subs(X, wp(z-z0,g2,g3))
)

rho_z_no_sqrt_d4

Eq(rho(z), -(-rho(0) + lambda[1])*(wp(z0, g2, g3) - wp(z1, g2, g3))/(-wp(z1, g2, g3) + wp(z - z0, g2, g3)) + lambda[1])

The modal power can also be written as follows:

In [35]:
gamma_uv_0 = Eq(gamma[j], solve(uv_j_rho,gamma[j])[0].subs(z,0))
rho_z0_lam1 = Eq(rho(z0), lam[1])
rho0_lam_1 = rho_z_sqrt_d4.subs(z,0).subs([gamma_uv_0.args]).subs(wp(-z0,g2,g3), wp(z0,g2,g3))
gamma_j_lam_1 = gamma_uv_0.subs([rho0_lam_1.args])
uv_uv_0_j = uv_j_rho.subs([
    gamma_uv_0.args, 
    rho_z_sqrt_d4.args, 
    rho_z_sqrt_d4.subs(z,0).args,
    (wp(-z0,g2,g3), wp(z0,g2,g3))
])
uv_mu_j_z1p = Eq(
    uv_uv_0_j.lhs - uv_uv_0_j.subs(z, mu[j]).subs(u(mu[j], mu[j]),0).lhs, 
    uv_uv_0_j.rhs - uv_uv_0_j.subs(z, mu[j]).subs(u(mu[j], mu[j]),0).rhs
)

rho_z0_lam1
rho0_lam_1
gamma_j_lam_1
uv_uv_0_j
uv_mu_j_z1p

Eq(rho(z0), lambda[1])

Eq(rho(0), lambda[1] + \wp\'(z1, g2, g3)/((-wp(z0, g2, g3) + wp(z1, g2, g3))*sqrt(d[4])))

Eq(gamma[j], u(0, mu[j])*v(0, mu[j]) + lambda[1] + \wp\'(z1, g2, g3)/((-wp(z0, g2, g3) + wp(z1, g2, g3))*sqrt(d[4])))

Eq(u(z, mu[j])*v(z, mu[j]), u(0, mu[j])*v(0, mu[j]) - \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(d[4])) + \wp\'(z1, g2, g3)/((-wp(z0, g2, g3) + wp(z1, g2, g3))*sqrt(d[4])))

Eq(u(z, mu[j])*v(z, mu[j]), \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sqrt(d[4])) - \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(d[4])))

The invariants $g_2, g_3$ can be expressed in terms of the parameters via substitution of symmetric polynomials in $\Omega_j$:

In [36]:
lam_Omegas = [Eq(lam[_j+2], solve(Omega_lams[_j],lam[_j+2])[0]).args for _j in range(3)]
qz_Omega = qz_lam.subs(lam_Omegas)
qz_Omega_b = Eq(qz_Omega.lhs, qz_Omega.rhs.expand().collect(q(z), factor))
dj_Omega = [
    Eq(
        qz_Omega_b.rhs.expand().coeff(q(z),_j).factor(), 
        drhop_d.subs([rho_q_lam1.args]).doit().rhs.expand().coeff(q(z), _j)
    ) for _j in range(4)
]

g2_dj = Eq(
    (
        g2_Omega.lhs - g2_Omega.rhs.doit().expand().simplify().doit().expand().simplify() +
        (dj_Omega[2].lhs**2/12 - dj_Omega[1].lhs*dj_Omega[3].lhs/4).expand().simplify() +
        dj_Omega[0].lhs*d[4]
    ).expand().simplify(),
    (
        (dj_Omega[2].rhs**2/12 - dj_Omega[1].rhs*dj_Omega[3].rhs/4).expand().simplify() +
        dj_Omega[0].rhs*d[4]
    ).expand().simplify()
)

g3_dj = Eq(
    (
        g3_Omega.lhs - g3_Omega.rhs.doit().expand().simplify().doit().expand().simplify() -
        (
            dj_Omega[2].lhs**3/216 - 
            dj_Omega[2].lhs*dj_Omega[3].lhs*dj_Omega[1].lhs/48  + 
            d[4]*dj_Omega[1].lhs**2/16 -
            dj_Omega[0].lhs*(8*d[2]*d[4] - 3*d[3]**2)/48
        )
    ).expand().simplify(),
    - (
        dj_Omega[2].rhs**3/216 - 
        dj_Omega[2].rhs*dj_Omega[3].rhs*dj_Omega[1].rhs/48  + 
        d[4]*dj_Omega[1].rhs**2/16 -
        dj_Omega[0].rhs*(8*d[2]*d[4] - 3*d[3]**2)/48
    ).expand().simplify()
)
g2_bcj = g2_dj.subs([_eq.args for _eq in d_j_coeffs]).expand().simplify()
g3_bcj = g3_dj.subs([_eq.args for _eq in d_j_coeffs]).expand().simplify()

qz_Omega
qz_Omega_b
for _eq in dj_Omega:
    _eq

g2_dj
g3_dj

# g2_bcj
# g3_bcj

Eq(Derivative(q(z), z)**2, (q(z) - 1/Omega[1])*(q(z) - 1/Omega[2])*(q(z) - 1/Omega[3])*q(z)*d[4])

Eq(Derivative(q(z), z)**2, -(Omega[1]*Omega[2] + Omega[1]*Omega[3] + Omega[2]*Omega[3])*q(z)**3*d[4]/(Omega[1]*Omega[2]*Omega[3]) + (Omega[1] + Omega[2] + Omega[3])*q(z)**2*d[4]/(Omega[1]*Omega[2]*Omega[3]) + q(z)**4*d[4] - q(z)*d[4]/(Omega[1]*Omega[2]*Omega[3]))

Eq(0, d[0] + d[1]*lambda[1] + d[2]*lambda[1]**2 + d[3]*lambda[1]**3 + d[4]*lambda[1]**4)

Eq(-d[4]/(Omega[1]*Omega[2]*Omega[3]), d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)

Eq((Omega[1] + Omega[2] + Omega[3])*d[4]/(Omega[1]*Omega[2]*Omega[3]), d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)

Eq(-(Omega[1]*Omega[2] + Omega[1]*Omega[3] + Omega[2]*Omega[3])*d[4]/(Omega[1]*Omega[2]*Omega[3]), d[3] + 4*d[4]*lambda[1])

Eq(g2, d[0]*d[4] - d[1]*d[3]/4 + d[2]**2/12)

Eq(g3, d[0]*d[2]*d[4]/6 - d[0]*d[3]**2/16 - d[1]**2*d[4]/16 + d[1]*d[2]*d[3]/48 - d[2]**3/216)

The point $z_1$ is then defined as:

In [37]:
one_over_Omega_j_w_wp = Eq(
    1/(Omega[j]),
    wpp(z1, g2, g3)/(wp(z1, g2, g3) - wp(omega[j], g2, g3))/sqrt(d[4])
)

one_over_Omega_j_w_wp_check_eq = one_over_Omega_j_w_wp.subs([(xi_j_root.rhs, xi_j_root.lhs), wp_z1.args])
one_over_Omega_j_w_wp_check = (
    one_over_Omega_j_w_wp_check_eq.lhs.subs(Omega[j], solve(xi_j_omega_sum_b, Omega[j])[0]) - 
    one_over_Omega_j_w_wp_check_eq.rhs*wpp_z1.rhs/wpp_z1.lhs
).doit().simplify() == 0
if not one_over_Omega_j_w_wp_check:
    raise ValueError('one_over_Omega_j_w_wp_check failed')
    
Omega_j_w_wp = Eq(1/one_over_Omega_j_w_wp.lhs, 1/one_over_Omega_j_w_wp.rhs)
    
Omega_j_subs = [Omega_j_w_wp.subs(j,_j+1).args for _j in range(3)]
wp_omega_sum = Eq(wp(omega[1], g2, g3) + wp(omega[2], g2, g3) + wp(omega[3], g2, g3), 0)
wp_omega_z1_prod = Eq(
    (wp(z1, g2, g3) - wp(omega[1], g2, g3))*
    (wp(z1, g2, g3) - wp(omega[2], g2, g3))*
    (wp(z1, g2, g3) - wp(omega[3], g2, g3)),
    (wpp(z1,g2,g3)**2)/4
)
wp_omega_g2 = Eq(
    wp(omega[1], g2, g3)*wp(omega[2], g2, g3) + 
    wp(omega[1], g2, g3)*wp(omega[3], g2, g3) + 
    wp(omega[2], g2, g3)*wp(omega[3], g2, g3),
    -g2/4
)

d_j_coeffs_wp = [
    _eq.subs(Omega_j_subs).simplify()
    .subs([wp_omega_sum.args, wp_omega_z1_prod.args]) 
    .expand().simplify()
    .subs([wp_omega_g2.args])
    .subs(
        2*(wp_omega_sum.lhs*wp(z1,g2,g3)).expand(), 
        2*(wp_omega_sum.rhs*wp(z1,g2,g3)).expand()
    )
    for _eq in dj_Omega
]

one_over_Omega_j_w_wp
for _eq in d_j_coeffs_wp:
    _eq


Eq(1/Omega[j], \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(omega[j], g2, g3))*sqrt(d[4])))

Eq(d[0] + d[1]*lambda[1] + d[2]*lambda[1]**2 + d[3]*lambda[1]**3 + d[4]*lambda[1]**4, 0)

Eq(4*\wp\'(z1, g2, g3)/sqrt(d[4]), -d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)

Eq(12*wp(z1, g2, g3), d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)

Eq(d[3] + 4*d[4]*lambda[1], 4*(g2/4 - 3*wp(z1, g2, g3)**2)*sqrt(d[4])/\wp\'(z1, g2, g3))

the point $z_0$ is defined as:

In [38]:
rho0_lam_1 = rho_z_sqrt_d4.subs(z,0).subs([gamma_uv_0.args]).subs(wp(-z0,g2,g3), wp(z0,g2,g3))
z1_wp_subs = [
    (wp(z1,g2,g3), solve(d_j_coeffs_wp[2], wp(z1,g2,g3))[0]),
    (wpp(z1,g2,g3), solve(d_j_coeffs_wp[1], wpp(z1,g2,g3))[0])
]
wp_z0_rho0_lam1 = Eq(
    wp(z0,g2,g3), 
    solve(rho0_lam_1, wp(z0,g2,g3))[0].expand().collect(wp(z1,g2,g3), factor)
    .subs(z1_wp_subs)
)
droh_0_z0 = Eq(
    diff(rho_z_sqrt_d4.lhs,z).subs(z,0).simplify(), 
    diff(rho_z_sqrt_d4.rhs,z).subs(z,0).simplify().subs(wp(-z0,g2,g3),wp(z0,g2,g3))*
    (-wpp(z0,g2,g3)/diff(wp(z-z0,g2,g3),z).subs(z,0).doit())
)
wpp_z0_rho0_lam1 = Eq(
    wpp(z0,g2,g3), 
    solve(droh_0_z0.subs(*wp_z0_rho0_lam1.args).subs(z1_wp_subs), wpp(z0,g2,g3))[0]
)


wp_z0_rho0_lam1
wpp_z0_rho0_lam1

Eq(wp(z0, g2, g3), d[2]/12 + d[3]*lambda[1]/4 + d[4]*lambda[1]**2/2 + (-d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)/(4*(-rho(0) + lambda[1])))

Eq(\wp\'(z0, g2, g3), (d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)*Subs(Derivative(rho(z), z), z, 0)/(4*(rho(0) - lambda[1])**2))

and the point $-z_0+\mu_j$ is given by:

In [39]:
rho_mu_j_gamma = Eq(rho(mu[j]), gamma[j])
wp_z0_mu_j_d = Eq(
    wp(mu[j]-z0,g2,g3), 
    solve(rho_z_sqrt_d4.subs([(z, mu[j]), rho_mu_j_gamma.args]), wp(mu[j]-z0,g2,g3))[0]
    .expand().collect(sqrt(d[4]), factor)
    .subs([
        (wp(z1,g2,g3), solve(d_j_coeffs_wp[2], wp(z1,g2,g3))[0]),
        (wpp(z1,g2,g3), solve(d_j_coeffs_wp[1], wpp(z1,g2,g3))[0])
    ])
)
rhop_mu_j_gamma = Eq(
    rhop(mu[j]), 
    Q(gamma[1] - gamma[j], gamma[2] - gamma[j], gamma[3] - gamma[j], gamma[4] - gamma[j])
)
rhop_mu_j_gamma_b = rhop_mu_j_gamma.subs([
    Q_poly_uv_b
    .subs([(u(z, mu[_j])*v(z, mu[_j]), gamma[_j] - rho(z)) for _j in range(1,5)])
    .subs(rho(z), gamma[j]).args,
    
])
drho_wp_d4_b_at_mu_j = (
    drho_wp_d4_b.subs(z, mu[j])
    .subs(diff(rho(mu[j]),mu[j]), rhop(mu[j]))
    .subs(z1_wp_subs)
    .subs(*wp_z0_mu_j_d.args)
    .subs(*rhop_mu_j_gamma_b.args)
)
wpp_z0_mu_j_d = Eq(wpp(mu[j]-z0,g2,g3), solve(drho_wp_d4_b_at_mu_j, wpp(mu[j]-z0,g2,g3))[0].factor())


rho_mu_j_gamma
wp_z0_mu_j_d

drho_wp_d4_b
rhop_mu_j_gamma
rhop_mu_j_gamma_b
wpp_z0_mu_j_d

Eq(rho(mu[j]), gamma[j])

Eq(wp(-z0 + mu[j], g2, g3), d[2]/12 + d[3]*lambda[1]/4 + d[4]*lambda[1]**2/2 - (-d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)/(4*(gamma[j] - lambda[1])))

Eq(Derivative(rho(z), z), \wp\'(z1, g2, g3)*\wp\'(z - z0, g2, g3)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))**2*sqrt(d[4])))

Eq(\\rho\'(mu[j]), Q(gamma[1] - gamma[j], gamma[2] - gamma[j], gamma[3] - gamma[j], gamma[4] - gamma[j]))

Eq(\\rho\'(mu[j]), b[0] + b[1]*gamma[j] + b[2]*gamma[j]**2)

Eq(\wp\'(-z0 + mu[j], g2, g3), -(b[0] + b[1]*gamma[j] + b[2]*gamma[j]**2)*(d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)/(4*(gamma[j] - lambda[1])**2))

Having solved for $\rho(z)$ we now define the function $\rho(z) - \rho(x)$:

In [40]:
rho_z_min_x = Eq(
    rho_z_sqrt_d4.lhs - rho_z_sqrt_d4.lhs.subs(z,x), 
    (rho_z_sqrt_d4.rhs - rho_z_sqrt_d4.rhs.subs(z,x))
)
rho_z_min_x

Eq(-rho(x) + rho(z), \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(d[4])) - \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(x - z0, g2, g3))*sqrt(d[4])))

From the unified theory, we know that $u(z, \mu_j), v(z, \mu_j)$ solve the following differential equations:

In [41]:
dphiz_unified = Eq(
    Derivative(phi(z, mu[j]), z),
    (Q(-rho(z) + gamma[1], -rho(z) + gamma[N]) - Q(gamma[1] - gamma[j], gamma[N] - gamma[j]))/
    (-2*rho(z) + 2*gamma[j]) - Derivative(Q(-rho(z) + gamma[1], -rho(z) + gamma[N]), gamma[j])
)
dphiz_unified_m4 = Eq(
    Derivative(phi(z, mu[m]), z),
    (
        Q(-rho(z) + gamma[1], -rho(z) + gamma[2], -rho(z) + gamma[3], -rho(z) + gamma[4]) - 
        Q(gamma[1] - gamma[m], gamma[2] - gamma[m], gamma[3] - gamma[m], gamma[4] - gamma[m])
    )/(-rho(z) + gamma[m])/2
    - Derivative(Q(-rho(z) + gamma[1], -rho(z) + gamma[2], -rho(z) + gamma[3], -rho(z) + gamma[4]), gamma[m])
)
duz_unified  = Eq(
    Derivative(u(z, mu[j]), z)/u(z, mu[j]),
    (rhop(z) - rhop(mu[j]))/(2*(rho(z) - rho(mu[j]))) + Derivative(phi(z, mu[j]), z)
)
dvz_unified  = Eq(
    Derivative(v(z, mu[j]), z)/v(z, mu[j]),
    (rhop(z) + rhop(mu[j]))/(2*(rho(z) - rho(mu[j]))) - Derivative(phi(z, mu[j]), z)
)

dphiz_unified
duz_unified
dvz_unified

Eq(Derivative(phi(z, mu[j]), z), (Q(-rho(z) + gamma[1], -rho(z) + gamma[N]) - Q(gamma[1] - gamma[j], gamma[N] - gamma[j]))/(-2*rho(z) + 2*gamma[j]) - Derivative(Q(-rho(z) + gamma[1], -rho(z) + gamma[N]), gamma[j]))

Eq(Derivative(u(z, mu[j]), z)/u(z, mu[j]), (\\rho\'(z) - \\rho\'(mu[j]))/(2*rho(z) - 2*rho(mu[j])) + Derivative(phi(z, mu[j]), z))

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]), (\\rho\'(z) + \\rho\'(mu[j]))/(2*rho(z) - 2*rho(mu[j])) - Derivative(phi(z, mu[j]), z))

We first derive expressions for the logarithmic derivative parts:

In [42]:
dwp_zx_subs = [(diff(wp(z,g2,g3),z), wpp(z,g2,g3)), (diff(wp(x,g2,g3),x), wpp(x,g2,g3))]
log_rho_z_x = Eq(
    log(rho(z) - rho(x)),
    log(-wp(x - z0, g2, g3) + wp(z - z0, g2, g3)) + 
    log(-wp(z1, g2, g3)/sqrt(d[4])) -
    log(-wp(z1, g2, g3) + wp(z - z0, g2, g3)) - 
    log(-wp(z1, g2, g3) + wp(x - z0, g2, g3))
)

dlog_z_min_x = Eq(
    diff(log_rho_z_x.lhs,z) - diff(log_rho_z_x.lhs,x),
    diff(log_rho_z_x.rhs.subs([(z, z+z0), (x, x+z0)]),z).subs(dwp_zx_subs).subs([(z, z-z0), (x, x-z0)]) - 
    diff(log_rho_z_x.rhs.subs([(z, z+z0), (x, x+z0)]),x).subs(dwp_zx_subs).subs([(z, z-z0), (x, x-z0)])
)
dlog_z_plus_x = Eq(
    diff(log_rho_z_x.lhs,z) + diff(log_rho_z_x.lhs,x),
    diff(log_rho_z_x.rhs.subs([(z, z+z0), (x, x+z0)]),z).subs(dwp_zx_subs).subs([(z, z-z0), (x, x-z0)]) + 
    diff(log_rho_z_x.rhs.subs([(z, z+z0), (x, x+z0)]),x).subs(dwp_zx_subs).subs([(z, z-z0), (x, x-z0)])
)

pw_to_zw_eqs = [
    pw_to_zw_identity_one_sided.subs([(z,t),(x,y)]).subs([(t,z-z0), (y, z1)]),
    pw_to_zw_identity_one_sided.subs([(z,t),(x,y)]).subs([(t,z-z0), (y, x - z0)]),
    pw_to_zw_identity_one_sided.subs([(z,t),(x,y)]).subs([(t,x-z0), (y, z - z0)]),
    pw_to_zw_identity_one_sided.subs([(z,t),(x,y)]).subs([(t,x-z0), (y, z1)])
]

dlog_z_min_x_zeta = Eq(
    (dlog_z_min_x.lhs/2).simplify(),
    ((
        dlog_z_min_x.rhs 
        - pw_to_zw_eqs[0].lhs + pw_to_zw_eqs[0].rhs
        + pw_to_zw_eqs[1].lhs - pw_to_zw_eqs[1].rhs
        - pw_to_zw_eqs[2].lhs + pw_to_zw_eqs[2].rhs
        + pw_to_zw_eqs[3].lhs - pw_to_zw_eqs[3].rhs
    ).collect([wpp(z-z0,g2,g3), wpp(x-z0,g2,g3)], simplify)/2).subs([
        (zeta(x-z,g2,g3), -zeta(z-x,g2,g3))
    ])
)
dlog_z_plus_x_zeta = Eq(
    (dlog_z_plus_x.lhs/2).simplify(),
    ((
        dlog_z_plus_x.rhs 
        - pw_to_zw_eqs[0].lhs + pw_to_zw_eqs[0].rhs
        + pw_to_zw_eqs[1].lhs - pw_to_zw_eqs[1].rhs
        + pw_to_zw_eqs[2].lhs - pw_to_zw_eqs[2].rhs
        - pw_to_zw_eqs[3].lhs + pw_to_zw_eqs[3].rhs
    ).collect([wpp(z-z0,g2,g3), wpp(x-z0,g2,g3)], simplify)/2).subs([
        (zeta(x-z,g2,g3), -zeta(z-x,g2,g3))
    ])
)
    
dlog_z_min_x
print()
dlog_z_plus_x
print()
dlog_z_min_x_zeta
print()
dlog_z_plus_x_zeta

Eq(Derivative(rho(x), x)/(-rho(x) + rho(z)) + Derivative(rho(z), z)/(-rho(x) + rho(z)), \wp\'(x - z0, g2, g3)/(-wp(x - z0, g2, g3) + wp(z - z0, g2, g3)) + \wp\'(z - z0, g2, g3)/(-wp(x - z0, g2, g3) + wp(z - z0, g2, g3)) - \wp\'(z - z0, g2, g3)/(-wp(z1, g2, g3) + wp(z - z0, g2, g3)) + \wp\'(x - z0, g2, g3)/(-wp(z1, g2, g3) + wp(x - z0, g2, g3)))

Eq(-Derivative(rho(x), x)/(-rho(x) + rho(z)) + Derivative(rho(z), z)/(-rho(x) + rho(z)), -\wp\'(x - z0, g2, g3)/(-wp(x - z0, g2, g3) + wp(z - z0, g2, g3)) + \wp\'(z - z0, g2, g3)/(-wp(x - z0, g2, g3) + wp(z - z0, g2, g3)) - \wp\'(z - z0, g2, g3)/(-wp(z1, g2, g3) + wp(z - z0, g2, g3)) - \wp\'(x - z0, g2, g3)/(-wp(z1, g2, g3) + wp(x - z0, g2, g3)))

Eq((-Derivative(rho(x), x) - Derivative(rho(z), z))/(2*(rho(x) - rho(z))), zeta(-x + z, g2, g3) + zeta(x - z0 - z1, g2, g3)/2 + zeta(x - z0 + z1, g2, g3)/2 - zeta(z - z0 - z1, g2, g3)/2 - zeta(z - z0 + z1, g2, g3)/2)

Eq((Derivative(rho(x), x) - Derivative(rho(z), z))/(2*(rho(x) - rho(z))), zeta(x + z - 2*z0, g2, g3) - zeta(x - z0 - z1, g2, g3)/2 - zeta(x - z0 + z1, g2, g3)/2 - zeta(z - z0 - z1, g2, g3)/2 - zeta(z - z0 + z1, g2, g3)/2)

Then we derive expressions for the gauge function parts:

In [43]:
## -- Difference part --

pjm = Eq(p[j,m], gamma[j] - gamma[m])

pjk_diff = Eq(
    Sum((p[j, m] + p[k, m])*a[j, k]/4, (j, 1, 4), (k, 1, 4)),
    Sum(gamma[j]*Sum(a[j, k], (k, 1, 4)), (j, 1, 4))/2
    - gamma[m]/2*Sum(a[j, k], (j, 1, 4), (k, 1, 4))
)
pjm_subs = [pjm.subs([(j,_j),(m,_m)]).args for _j in range(1,5) for _m in range(1,5)]
pjk_diff_checks = [
    not pjk_diff.doit().expand().simplify().subs(m, _m).subs(ajk_syms).subs(pjm_subs).simplify() 
    for _m in range(1,5)
] 
if any(pjk_diff_checks):
    raise ValueError('pjk_diff_checks failed')

rho_a_p_gamma_min_const = Eq(
    (
        Q(
            -rho(z) + gamma[1], 
            -rho(z) + gamma[2], 
            -rho(z) + gamma[3], 
            -rho(z) + gamma[4]
        ) - 
        Q(
            gamma[1] - gamma[m], 
            gamma[2] - gamma[m], 
            gamma[3] - gamma[m], 
            gamma[4] - gamma[m]
        )
    )/(gamma[m] - rho(z))/2,
    Sum(a[j], (j, 1, 4))/2 +
    - rho(z)/4*Sum(a[j, k], (j, 1, 4), (k, 1, 4)) 
    + gamma[m]/4*Sum(a[j, k], (j, 1, 4), (k, 1, 4)) 
    + Sum((p[j, m] + p[k, m])*a[j, k]/4, (j, 1, 4), (k, 1, 4))
)
rho_a_p_gamma_min_const = rho_a_p_gamma_min_const.subs(*pjk_diff.args)

Q_poly_gam_rho = Q_poly_uv.subs([(u(z,mu[_j])*v(z,mu[_j]), gamma[_j] - rho(z)) for _j in range(1,5)])
Q_gam_minus = Eq(
    (Q_poly_gam_rho.lhs - Q_poly_gam_rho.lhs.subs(rho(z), gamma[m]))/(gamma[m] - rho(z))/2,
    (Q_poly_gam_rho.rhs - Q_poly_gam_rho.rhs.subs(rho(z), gamma[m]))/(gamma[m] - rho(z))/2
)

Q_gam_checks = [
    (
        rho_a_p_gamma_min_const.subs(m,_m).doit().subs(pjm_subs).subs(ajk_syms).expand().simplify().rhs -
        Q_gam_minus.subs(m,_m).doit().subs(ajk_syms).expand().simplify().rhs
    ) != 0
    for _m in range(1,4)
]
if any(Q_gam_checks):
    raise ValueError('Q_gam_checks failed')
    
    
## -- Derivative part -- 

dQ_rho_a_p_gamma = Eq(
    Derivative(
        Q(
            -rho(z) + gamma[1], 
            -rho(z) + gamma[2], 
            -rho(z) + gamma[3], 
            -rho(z) + gamma[4]
        )
        ,gamma[m]
    ),
    a[m] + Sum(gamma[k]*a[m, k], (k, 1, 4)) - rho(z)*Sum(a[m, k], (k, 1, 4))
)
dQ_checks = [
    (
        diff(Q_poly_gam_rho.rhs.doit(),gamma[_m]).subs(ajk_syms) - 
        dQ_rho_a_p_gamma.rhs.subs(m,_m).doit().subs(ajk_syms)
    ).simplify() != 0
    for _m in range(1,5)
]
if any(dQ_checks):
    raise ValueError('dQ_checks failed')
    
    
## -- Phi derivative --

dphi_m_Q = dphiz_unified_m4.subs([rho_a_p_gamma_min_const.args, dQ_rho_a_p_gamma.args])
dphi_m_Q = Eq(dphi_m_Q.lhs, dphi_m_Q.rhs.collect(rho(z)))
Lams = [Eq(Lam[_j, m], dphi_m_Q.rhs.coeff(rho(z),_j) ) for _j in range(2)]
dphi_m_Q_Lam = dphi_m_Q.subs([(_eq.rhs, _eq.lhs) for _eq in Lams])


rho_a_p_gamma_min_const
dQ_rho_a_p_gamma
dphi_m_Q
for _eq in Lams:
    _eq
dphi_m_Q_Lam

Eq((Q(-rho(z) + gamma[1], -rho(z) + gamma[2], -rho(z) + gamma[3], -rho(z) + gamma[4]) - Q(gamma[1] - gamma[m], gamma[2] - gamma[m], gamma[3] - gamma[m], gamma[4] - gamma[m]))/(2*(-rho(z) + gamma[m])), -rho(z)*Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4 - gamma[m]*Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4 + Sum(gamma[j]*Sum(a[j, k], (k, 1, 4)), (j, 1, 4))/2 + Sum(a[j], (j, 1, 4))/2)

Eq(Derivative(Q(-rho(z) + gamma[1], -rho(z) + gamma[2], -rho(z) + gamma[3], -rho(z) + gamma[4]), gamma[m]), -rho(z)*Sum(a[m, k], (k, 1, 4)) + a[m] + Sum(a[m, k]*gamma[k], (k, 1, 4)))

Eq(Derivative(phi(z, mu[m]), z), (Sum(a[m, k], (k, 1, 4)) - Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4)*rho(z) - a[m] - gamma[m]*Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4 - Sum(a[m, k]*gamma[k], (k, 1, 4)) + Sum(gamma[j]*Sum(a[j, k], (k, 1, 4)), (j, 1, 4))/2 + Sum(a[j], (j, 1, 4))/2)

Eq(Lambda[0, m], -a[m] - gamma[m]*Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4 - Sum(a[m, k]*gamma[k], (k, 1, 4)) + Sum(gamma[j]*Sum(a[j, k], (k, 1, 4)), (j, 1, 4))/2 + Sum(a[j], (j, 1, 4))/2)

Eq(Lambda[1, m], Sum(a[m, k], (k, 1, 4)) - Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4)

Eq(Derivative(phi(z, mu[m]), z), rho(z)*Lambda[1, m] + Lambda[0, m])

We now put these parts together to solve for $u,v$:

In [44]:
## -- Build zeta expansion for log diffs of u, v --

duz_unified_zeta = duz_unified.expand().subs([
    dlog_z_plus_x_zeta.subs([
        (diff(rho(z),z),rhop(z)), (diff(rho(x),x),rhop(x))
    ]).subs(x, mu[j]).simplify().expand().args,
    dphi_m_Q_Lam.subs([(m,j), rho_zeta.args]).args
])
dvz_unified_zeta = dvz_unified.expand().subs([
    dlog_z_min_x_zeta.subs([
        (diff(rho(z),z),rhop(z)), (diff(rho(x),x),rhop(x))
    ]).subs(x, mu[j]).simplify().expand().args,
    dphi_m_Q_Lam.subs([(m,j), rho_zeta.args]).args
])
duz_unified_zeta = Eq(
    duz_unified_zeta.lhs, 
    duz_unified_zeta.rhs.expand().collect([Lam[1,j]/sqrt(d[4])]).subs(zeta(-z+z1+z0,g2,g3), -zeta(z-z1-z0,g2,g3))
)
dvz_unified_zeta = Eq(
    dvz_unified_zeta.lhs, 
    dvz_unified_zeta.rhs.expand().collect([Lam[1,j]/sqrt(d[4])]).subs(zeta(-z+z1+z0,g2,g3), -zeta(z-z1-z0,g2,g3))
)


## -- Convert zetas to log diff sigmas --

zw_dlog_subs = [
    zw_dlog_sigma_identity.subs([(x, z1)]).args,
    zw_dlog_sigma_identity.subs([(x, mu[j])]).args,
    zw_dlog_sigma_identity.subs([(x, z0 - z1)]).args,
    zw_dlog_sigma_identity.subs([(x, z0 + z1)]).args,
    zw_dlog_sigma_identity.subs([(x, 2*z0 - mu[j])]).args
]

duz_unified_zeta_b = duz_unified_zeta.subs(zw_dlog_subs)
dvz_unified_zeta_b = dvz_unified_zeta.subs(zw_dlog_subs)


## -- Combine log diff sigmas --

dlog_sub_d4 = Eq(
    Derivative(log(sigma(z - z0 + z1, g2, g3)), z)*Lam[1, j]/sqrt(d[4])
    -Derivative(log(sigma(z - z1 - z0, g2, g3)), z)*Lam[1, j]/sqrt(d[4]),
    Derivative(log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3)), z)*Lam[1, j]/sqrt(d[4])
)
dlog_sub_no_d4_1 = Eq(
    Derivative(log(sigma(z - 2*z0 + mu[j], g2, g3)), z)
    - Derivative(log(sigma(z - z1 - z0, g2, g3)), z)/2
    - Derivative(log(sigma(z - z0 + z1, g2, g3)), z)/2,
    Derivative(log(
        sigma(z - 2*z0 + mu[j], g2, g3)/
        sqrt(sigma(z - z1 - z0, g2, g3)*sigma(z - z0 + z1, g2, g3))
    ), z)
)
dlog_sub_no_d4_2 =Eq(
    Derivative(log(sigma(z - mu[j], g2, g3)), z)
    - Derivative(log(sigma(z - z1 - z0, g2, g3)), z)/2
    - Derivative(log(sigma(z - z0 + z1, g2, g3)), z)/2,
    Derivative(log(
        sigma(z - mu[j], g2, g3)/
        sqrt(sigma(z - z1 -z0, g2, g3)*sigma(z - z0 + z1, g2, g3))
    ), z)
)
_log_combined_subs = [dlog_sub_d4.args, dlog_sub_no_d4_1.args, dlog_sub_no_d4_2.args]

duz_unified_zeta_c = duz_unified_zeta_b.expand().subs(_log_combined_subs)
dvz_unified_zeta_c = dvz_unified_zeta_b.expand().subs(_log_combined_subs)


## -- Substitute short param names for non-log parts --

_duz_ts = duz_unified_zeta_b.expand().rhs.args
duz_unified_zeta_b_non_log_terms = sum(_t for _t in _duz_ts if 'log' not in str(_t))
r0j_non_log_part = Eq(r[0,j], duz_unified_zeta_b_non_log_terms)
r1j_log_part = Eq(r[1,j], Lam[1,j]/sqrt(d[4]))
r2j_log_part = Eq(r[2,j], Lam[1,j]/sqrt(d[4]) - Rational(1,2))

duz_unified_zeta_c = Eq(
    duz_unified_zeta_c.lhs, 
    (duz_unified_zeta_c.rhs - r0j_non_log_part.rhs + r0j_non_log_part.lhs)
    .subs(sqrt(d[4]), solve(r1j_log_part, sqrt(d[4]))[0])
)
dvz_unified_zeta_c = Eq(
    dvz_unified_zeta_c.lhs, 
    (dvz_unified_zeta_c.rhs + r0j_non_log_part.rhs - r0j_non_log_part.lhs)
    .subs(sqrt(d[4]), solve(r1j_log_part, sqrt(d[4]))[0])
)


## -- Define the solutions after integrating --

uz_sol = Eq(
    u(z, mu[j]),
    sigma(z - 2*z0 + mu[j], g2, g3)/sqrt(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3))*
    exp(log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + r[0, j]*z + r[3,j]/2 + epsilon[j])
)
vz_sol = Eq(
    v(z, mu[j]),
    sigma(z - mu[j], g2, g3)/sqrt(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3))*
    exp(-log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - r[0, j]*z + r[3,j]/2 - epsilon[j])
)

uz_sol_no_sqrt = Eq(
    u(z, mu[j]),
    sigma(z - 2*z0 + mu[j], g2, g3)/sigma(z - z0 - z1, g2, g3)*
    exp(log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[2, j] + r[0, j]*z + r[3,j]/2 + epsilon[j])
)
vz_sol_no_sqrt = Eq(
    v(z, mu[j]),
    sigma(z - mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)*
    exp(-log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[2, j] - r[0, j]*z + r[3,j]/2 - epsilon[j])
)

uz_sol_frac_exp = Eq(
    u(z, mu[j]),
    sigma(z - 2*z0 + mu[j], g2, g3)*
    sigma(z - z0 + z1, g2, g3)**r[2,j]*
    sigma(z - z0 - z1, g2, g3)**(-r[2,j]-1)*
    exp(r[0, j]*z + r[3,j]/2 + epsilon[j])
)
vz_sol_frac_exp = Eq(
    v(z, mu[j]),
    sigma(z - mu[j], g2, g3)*
    sigma(z - z0 - z1, g2, g3)**r[2,j]*
    sigma(z - z0 + z1, g2, g3)**(-r[2,j]-1)*
    exp( - r[0, j]*z + r[3,j]/2 - epsilon[j])
)

## -- Check they solve the equation we started from --

zw_dlog_sigma_id_b = Eq(zw_dlog_sigma_identity.rhs, zw_dlog_sigma_identity.lhs)
sig_zw_subs = [zw_dlog_sigma_id_b.subs(x,_x).doit().args for _x in [z0 - z1, z0 + z1, 2*z0 - mu[j], mu[j]]]
r0r1r2_subs = [r0j_non_log_part.args, r1j_log_part.args, r2j_log_part.args]

u_sol_check = (
    duz_unified_zeta.lhs.subs([uz_sol.args]).doit().simplify().subs(sig_zw_subs).subs(r0r1r2_subs)
    - duz_unified_zeta.rhs
).simplify() == 0
v_sol_check = (
    dvz_unified_zeta.lhs.subs([vz_sol.args]).doit().simplify().subs(sig_zw_subs).subs(r0r1r2_subs)
    - dvz_unified_zeta.rhs
).simplify() == 0
if not u_sol_check:
    raise ValueError('u_sol_check failed')
if not v_sol_check:
    raise ValueError('v_sol_check failed')
    
u_sol_no_sqrt_check = (
    duz_unified_zeta.lhs.subs([uz_sol_no_sqrt.args]).doit().simplify().subs(sig_zw_subs).subs(r0r1r2_subs)
    - duz_unified_zeta.rhs
).simplify() == 0
v_sol_no_sqrt_check = (
    dvz_unified_zeta.lhs.subs([vz_sol_no_sqrt.args]).doit().simplify().subs(sig_zw_subs).subs(r0r1r2_subs)
    - dvz_unified_zeta.rhs
).simplify() == 0
if not u_sol_no_sqrt_check:
    raise ValueError('u_sol_no_sqrt_check failed')
if not v_sol_no_sqrt_check:
    raise ValueError('v_sol_no_sqrt_check failed')
    
u_sol_frac_exp_check = (
    (duz_unified_zeta.lhs.subs([uz_sol_frac_exp.args]).doit()
    .simplify().subs(r0r1r2_subs).expand().subs(sig_zw_subs)
    ).simplify().subs(sig_zw_subs)
    - duz_unified_zeta.rhs
).simplify() == 0
v_sol_frac_exp_check = (
    (dvz_unified_zeta.lhs.subs([vz_sol_frac_exp.args]).doit()
    .simplify().subs(r0r1r2_subs).expand().subs(sig_zw_subs)
    ).simplify().subs(sig_zw_subs)
    - dvz_unified_zeta.rhs
).simplify() == 0
if not u_sol_frac_exp_check:
    raise ValueError('u_sol_frac_exp_check failed')
if not v_sol_frac_exp_check:
    raise ValueError('v_sol_frac_exp_check failed')


duz_unified_zeta
print()
dvz_unified_zeta
print()

duz_unified_zeta_b
print()
dvz_unified_zeta_b
print()

r0j_non_log_part
r1j_log_part
r2j_log_part
print()

duz_unified_zeta_c
print()
dvz_unified_zeta_c
print()

uz_sol
vz_sol
print()
uz_sol_no_sqrt
vz_sol_no_sqrt
print()
uz_sol_frac_exp
vz_sol_frac_exp

Eq(Derivative(u(z, mu[j]), z)/u(z, mu[j]), (-2*zeta(z1, g2, g3) - zeta(z - z0 - z1, g2, g3) + zeta(z - z0 + z1, g2, g3))*Lambda[1, j]/sqrt(d[4]) + zeta(z - 2*z0 + mu[j], g2, g3) - zeta(z - z0 - z1, g2, g3)/2 - zeta(z - z0 + z1, g2, g3)/2 - zeta(-z0 - z1 + mu[j], g2, g3)/2 - zeta(-z0 + z1 + mu[j], g2, g3)/2 + Lambda[0, j] + Lambda[1, j]*lambda[1])

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]), (2*zeta(z1, g2, g3) + zeta(z - z0 - z1, g2, g3) - zeta(z - z0 + z1, g2, g3))*Lambda[1, j]/sqrt(d[4]) + zeta(z - mu[j], g2, g3) - zeta(z - z0 - z1, g2, g3)/2 - zeta(z - z0 + z1, g2, g3)/2 + zeta(-z0 - z1 + mu[j], g2, g3)/2 + zeta(-z0 + z1 + mu[j], g2, g3)/2 - Lambda[0, j] - Lambda[1, j]*lambda[1])

Eq(Derivative(u(z, mu[j]), z)/u(z, mu[j]), (-2*zeta(z1, g2, g3) - Derivative(log(sigma(z - z0 - z1, g2, g3)), z) + Derivative(log(sigma(z - z0 + z1, g2, g3)), z))*Lambda[1, j]/sqrt(d[4]) - zeta(-z0 - z1 + mu[j], g2, g3)/2 - zeta(-z0 + z1 + mu[j], g2, g3)/2 + Derivative(log(sigma(z - 2*z0 + mu[j], g2, g3)), z) - Derivative(log(sigma(z - z0 - z1, g2, g3)), z)/2 - Derivative(log(sigma(z - z0 + z1, g2, g3)), z)/2 + Lambda[0, j] + Lambda[1, j]*lambda[1])

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]), (2*zeta(z1, g2, g3) + Derivative(log(sigma(z - z0 - z1, g2, g3)), z) - Derivative(log(sigma(z - z0 + z1, g2, g3)), z))*Lambda[1, j]/sqrt(d[4]) + zeta(-z0 - z1 + mu[j], g2, g3)/2 + zeta(-z0 + z1 + mu[j], g2, g3)/2 + Derivative(log(sigma(z - mu[j], g2, g3)), z) - Derivative(log(sigma(z - z0 - z1, g2, g3)), z)/2 - Derivative(log(sigma(z - z0 + z1, g2, g3)), z)/2 - Lambda[0, j] - Lambda[1, j]*lambda[1])

Eq(r[0, j], -2*zeta(z1, g2, g3)*Lambda[1, j]/sqrt(d[4]) - zeta(-z0 - z1 + mu[j], g2, g3)/2 - zeta(-z0 + z1 + mu[j], g2, g3)/2 + Lambda[0, j] + Lambda[1, j]*lambda[1])

...

Eq(v(z, mu[j]), sigma(z - mu[j], g2, g3)*exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - epsilon[j] + r[3, j]/2)/sqrt(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3)))

Eq(u(z, mu[j]), sigma(z - 2*z0 + mu[j], g2, g3)*exp(z*r[0, j] + log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[2, j] + epsilon[j] + r[3, j]/2)/sigma(z - z0 - z1, g2, g3))

Eq(v(z, mu[j]), sigma(z - mu[j], g2, g3)*exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[2, j] - epsilon[j] + r[3, j]/2)/sigma(z - z0 + z1, g2, g3))

Eq(u(z, mu[j]), sigma(z - 2*z0 + mu[j], g2, g3)*sigma(z - z0 - z1, g2, g3)**(-r[2, j] - 1)*sigma(z - z0 + z1, g2, g3)**r[2, j]*exp(z*r[0, j] + epsilon[j] + r[3, j]/2))

Eq(v(z, mu[j]), sigma(z - mu[j], g2, g3)*sigma(z - z0 - z1, g2, g3)**r[2, j]*sigma(z - z0 + z1, g2, g3)**(-r[2, j] - 1)*exp(-z*r[0, j] - epsilon[j] + r[3, j]/2))

From these, we can derive expressions for the product of $u,v$:

In [45]:
dlog_uv = Eq(
    duz_unified_zeta_b.lhs + dvz_unified_zeta_b.lhs, 
    (duz_unified_zeta_b.rhs + dvz_unified_zeta_b.rhs).expand()
)
dlog_uv_b = Eq(
    Derivative(log(u(z, mu[j])*v(z, mu[j])), z),
    Derivative(log(
        sigma(z - 2*z0 + mu[j], g2, g3)*sigma(z - mu[j], g2, g3)/
        (sigma(z - z1 - z0, g2, g3)*sigma(z - z0 + z1, g2, g3))
    ), z)
)

dlog_uv_check_lhs = (dlog_uv.lhs - dlog_uv_b.lhs).doit().simplify() == 0
dlog_uv_check_rhs = (dlog_uv.rhs - dlog_uv_b.rhs).doit().simplify() == 0
dlog_uv_check = dlog_uv_check_lhs and dlog_uv_check_rhs
if not dlog_uv_check:
    raise ValueError('dlog_uv_check failed')
    
uv_sol = Eq(
    u(z, mu[j])*v(z, mu[j]),
    sigma(z - mu[j], g2, g3)*sigma(z - 2*z0 + mu[j], g2, g3)/
    (sigma(z - z1 - z0, g2, g3)*sigma(z - z0 + z1, g2, g3))*exp(r[3,j])
)

dlog_uv
dlog_uv_b
uv_sol

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]) + Derivative(u(z, mu[j]), z)/u(z, mu[j]), Derivative(log(sigma(z - mu[j], g2, g3)), z) + Derivative(log(sigma(z - 2*z0 + mu[j], g2, g3)), z) - Derivative(log(sigma(z - z0 - z1, g2, g3)), z) - Derivative(log(sigma(z - z0 + z1, g2, g3)), z))

Eq(Derivative(log(u(z, mu[j])*v(z, mu[j])), z), Derivative(log(sigma(z - mu[j], g2, g3)*sigma(z - 2*z0 + mu[j], g2, g3)/(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3))), z))

Eq(u(z, mu[j])*v(z, mu[j]), sigma(z - mu[j], g2, g3)*sigma(z - 2*z0 + mu[j], g2, g3)*exp(r[3, j])/(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3)))

Which, when compared to the original expression for $u(z, \mu_j)v(z, \mu_j)$, fixes the constant $r_{2,j}$:

In [46]:
# -- Convert sigma sol to wp form --

sig_pw_z0_mu_j_z1 = Eq(
    sigma_p_identity.rhs.subs([(x,z-z0),(y, mu[j] - z0)])/
    sigma_p_identity.rhs.subs([(x,z-z0),(y, z1)]),
    sigma_p_identity.lhs.subs([(x,z-z0),(y, mu[j] - z0)])/
    sigma_p_identity.lhs.subs([(x,z-z0),(y, z1)])
)
uv_sol_wp = Eq(uv_sol.lhs, uv_sol.rhs * sig_pw_z0_mu_j_z1.rhs/sig_pw_z0_mu_j_z1.lhs)


## -- Compare with original form that used mu_j and solve for r2j --

r3j_conistency = Eq(
    uv_mu_j_z1p.lhs.simplify()/uv_sol_wp.lhs, 
    uv_mu_j_z1p.rhs.simplify()/uv_sol_wp.rhs
)
exp_r_3_j = Eq(exp(r[3,j]), solve(r3j_conistency, exp(r[3,j]))[0])
exp_r_3_j_sigma = exp_r_3_j.subs([
    (
        -sigma_p_identity.subs([(x,z1),(y, mu[j] - z0)]).lhs,
        -sigma_p_identity.subs([(x,z1),(y, mu[j] - z0)]).rhs
    ),
    pwp_sigma_dbl_ratio.subs(z,z1).args
])
exp_r_3_j_over_2 = Eq(
    exp(r[3,j]/2), 
    sqrt(wpp(z1, g2, g3))/sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))/d[4]**Rational(1,4) *
    sigma(z1, g2, g3) / sigma(-z0 + mu[j], g2, g3)
)
uv_mu_j_z0 = uv_mu_j_z1p.subs([(z,z0), (1/(-wp(0, g2, g3) + wp(z1, g2, g3)),0)])
uv_mu_j_z_at_0 = uv_uv_0_j.subs([(z, mu[j]),(u(mu[j], mu[j])*v(mu[j], mu[j]),0)])

exp_r_3_j
exp_r_3_j_sigma
exp_r_3_j_over_2
uv_mu_j_z0
uv_mu_j_z_at_0

Eq(exp(r[3, j]), \wp\'(z1, g2, g3)*sigma(z1, g2, g3)**2/((wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(-z0 + mu[j], g2, g3)**2*sqrt(d[4])))

Eq(exp(r[3, j]), sigma(2*z1, g2, g3)/(sigma(-z0 + z1 + mu[j], g2, g3)*sigma(z0 + z1 - mu[j], g2, g3)*sqrt(d[4])))

Eq(exp(r[3, j]/2), sqrt(\wp\'(z1, g2, g3))*sigma(z1, g2, g3)/(sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(-z0 + mu[j], g2, g3)*d[4]**(1/4)))

Eq(u(z0, mu[j])*v(z0, mu[j]), \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sqrt(d[4])))

Eq(0, u(0, mu[j])*v(0, mu[j]) - \wp\'(z1, g2, g3)/((wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sqrt(d[4])) + \wp\'(z1, g2, g3)/((-wp(z0, g2, g3) + wp(z1, g2, g3))*sqrt(d[4])))

and this constant leads to these formulas for $u,v$:

In [47]:
uz_sol_no_sqrt_b = uz_sol_no_sqrt.expand().subs([exp_r_3_j_over_2.args]).simplify()
vz_sol_no_sqrt_b = vz_sol_no_sqrt.expand().subs([exp_r_3_j_over_2.args]).simplify()

u_sqrt_wp_z0_z1 = Eq(
    u(z, mu[j]),
    d[4]**Rational(-1, 4)*
    sqrt(wpp(z1, g2, g3))/
    sqrt((wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3)))/
    sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))
    *sigma(z - 2*z0 + mu[j], g2, g3)/(sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3))
    *exp(z*r[0, j] + log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + epsilon[j])
)
v_sqrt_wp_z0_z1 = Eq(
    v(z, mu[j]),
    d[4]**Rational(-1, 4)*
    sqrt(wpp(z1, g2, g3))/
    sqrt((wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3)))/
    sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))
    *sigma(z - mu[j], g2, g3)/(sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3))
    *exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - epsilon[j])
)

uz_sol_no_sqrt_b
vz_sol_no_sqrt_b
u_sqrt_wp_z0_z1
v_sqrt_wp_z0_z1

Eq(u(z, mu[j]), sqrt(\wp\'(z1, g2, g3))*sigma(z1, g2, g3)*sigma(z - 2*z0 + mu[j], g2, g3)*exp(z*r[0, j] + log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[2, j] + epsilon[j])/(sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(-z0 + mu[j], g2, g3)*sigma(z - z0 - z1, g2, g3)*d[4]**(1/4)))

Eq(v(z, mu[j]), sqrt(\wp\'(z1, g2, g3))*sigma(z1, g2, g3)*sigma(z - mu[j], g2, g3)*exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[2, j] - epsilon[j])/(sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(-z0 + mu[j], g2, g3)*sigma(z - z0 + z1, g2, g3)*d[4]**(1/4)))

Eq(u(z, mu[j]), sqrt(\wp\'(z1, g2, g3))*sigma(z - 2*z0 + mu[j], g2, g3)*exp(z*r[0, j] + log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + epsilon[j])/(sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*d[4]**(1/4)))

Eq(v(z, mu[j]), sqrt(\wp\'(z1, g2, g3))*sigma(z - mu[j], g2, g3)*exp(-z*r[0, j] - log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - epsilon[j])/(sqrt(wp(z1, g2, g3) - wp(z - z0, g2, g3))*sqrt(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sigma(z - z0, g2, g3)*sigma(-z0 + mu[j], g2, g3)*d[4]**(1/4)))

In [48]:
uv_wp_z0z1muj = Eq(uz_sol_no_sqrt_b.lhs*vz_sol_no_sqrt_b.lhs,
    (uz_sol_no_sqrt_b.rhs*vz_sol_no_sqrt_b.rhs).simplify()*(
        sigma_p_identity.subs([(x,z1),(y, z - z0)]).rhs/
        sigma_p_identity.subs([(x,z1),(y, z - z0)]).lhs
    ).subs([
        (sigma(-z+z0+z1,g2,g3), -sigma(z-z0-z1,g2,g3))
    ])*(
        sigma_p_identity.subs([(x,mu[j]-z0),(y, z - z0)]).lhs/
        sigma_p_identity.subs([(x,mu[j]-z0),(y, z - z0)]).rhs
    ).subs([
        (sigma(-z+mu[j],g2,g3), -sigma(z-mu[j],g2,g3))
    ])
)

uv0_check = uv_mu_j_z_at_0.subs(*uv_wp_z0z1muj.subs(z,0).args).subs(wp(-z0,g2,g3), wp(z0,g2,g3)).simplify()
if not uv0_check:
    raise ValueError('uv0_check failed')

uv_wp_z0z1muj


Eq(u(z, mu[j])*v(z, mu[j]), (wp(z - z0, g2, g3) - wp(-z0 + mu[j], g2, g3))*\wp\'(z1, g2, g3)/((-wp(z1, g2, g3) + wp(z - z0, g2, g3))*(wp(z1, g2, g3) - wp(-z0 + mu[j], g2, g3))*sqrt(d[4])))

We can derive an expression for the ratio $\frac{u}{v}$:

In [49]:
dlog_u_over_v = Eq(
    duz_unified_zeta_b.lhs - dvz_unified_zeta_b.lhs,
    (duz_unified_zeta_b.rhs - dvz_unified_zeta_b.rhs).collect(Lam[1,j]/sqrt(d[4]), factor)
)
combined_dlog = Eq(
    - Derivative(log(sigma(z - z1 - z0, g2, g3)), z) + Derivative(log(sigma(z - z0 + z1, g2, g3)), z),
    Derivative(log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3)), z)
)
dlog_u_over_v_id = Eq(
    diff(log(u(z,mu[j])/v(z,mu[j])),z).doit().expand(),
    Derivative(log(u(z,mu[j])/v(z,mu[j])),z)
)

dlog_u_over_v_b = dlog_u_over_v.subs([
    combined_dlog.args, 
    combined_dlog.subs(z1,mu[j]).args,
    dlog_u_over_v_id.args,
]).expand()

dlog_u_over_v_b = Eq(
    dlog_u_over_v_b.lhs, 
    (dlog_u_over_v_b.rhs - 2*r0j_non_log_part.rhs + 2*r0j_non_log_part.lhs)
    .subs(sqrt(d[4]), solve(r1j_log_part, sqrt(d[4]))[0])
)

log_u_over_v_j = Eq(
    log(u(z, mu[j])/v(z, mu[j])),
    log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j]*2 + 
    log(sigma(z - 2*z0 + mu[j], g2, g3)/sigma(z - mu[j], g2, g3)) + 
    r[0, j]*2*z + r[3,j]
)
u_over_v_j = Eq(
    u(z, mu[j])/v(z, mu[j]),
    sigma(z - 2*z0 + mu[j], g2, g3)/sigma(z - mu[j], g2, g3)*
    exp(log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*2*r[1, j] + 2*r[0, j]*z + 2*epsilon[j])
)

u_over_v_check = (dlog_u_over_v_b.lhs.subs(*u_over_v_j.args) - dlog_u_over_v_b.rhs).doit().simplify() == 0
if not u_over_v_check:
    u_over_v_check
    raise ValueError('u_over_v_check failed')
    
u_over_v_check_2 = dlog_u_over_v_b.subs(*u_over_v_j.args).doit().simplify()
if not u_over_v_check_2:
    u_over_v_check
    raise ValueError('u_over_v_check_2 failed')


# dlog_u_over_v
print()
# r0j_non_log_part
# r1j_log_part
dlog_u_over_v_b
# log_u_over_v_j
u_over_v_j

Eq(Derivative(log(u(z, mu[j])/v(z, mu[j])), z), 2*Derivative(log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3)), z)*r[1, j] - Derivative(log(sigma(z - mu[j], g2, g3)), z) + Derivative(log(sigma(z - 2*z0 + mu[j], g2, g3)), z) + 2*r[0, j])

Eq(u(z, mu[j])/v(z, mu[j]), sigma(z - 2*z0 + mu[j], g2, g3)*exp(2*z*r[0, j] + 2*log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + 2*epsilon[j])/sigma(z - mu[j], g2, g3))

and from the product and ratio formulas we get the expressions for the squares:

In [50]:
u_sqrd = Eq(uv_sol.lhs*u_over_v_j.lhs, (uv_sol.rhs*u_over_v_j.rhs).simplify())
v_sqrd = Eq(uv_sol.lhs/u_over_v_j.lhs, (uv_sol.rhs/u_over_v_j.rhs).simplify())

u_sqrd_check = Eq(u_sqrd.lhs/uz_sol.lhs**2, (u_sqrd.rhs/uz_sol.rhs**2).simplify())
v_sqrd_check = Eq(v_sqrd.lhs/vz_sol.lhs**2, (v_sqrd.rhs/vz_sol.rhs**2).simplify())

if not u_sqrd_check:
    raise ValueError('u_sqrd_check failed')
if not v_sqrd_check:
    raise ValueError('v_sqrd_check failed')

u_sqrd
v_sqrd

Eq(u(z, mu[j])**2, sigma(z - 2*z0 + mu[j], g2, g3)**2*exp(2*z*r[0, j] + 2*log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + 2*epsilon[j] + r[3, j])/(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3)))

Eq(v(z, mu[j])**2, sigma(z - mu[j], g2, g3)**2*exp(-2*z*r[0, j] - 2*log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] - 2*epsilon[j] + r[3, j])/(sigma(z - z0 - z1, g2, g3)*sigma(z - z0 + z1, g2, g3)))

### Special Values

The below special values seem to suggest $r_1$ is a half integer which would be great if true but by its current definition this is not true. Need to find the mistake.

In [51]:
u_over_v_j_at_0 = u_over_v_j.subs(z,0)
u_over_v_j_at_z0 = u_over_v_j.subs(z,z0).subs([
    (sigma(-z0+mu[j], g2,g3), -sigma(z0-mu[j], g2,g3)),
    (sigma(-z1, g2,g3), -sigma(z1, g2,g3))
])
u_over_v_j_shifted_min_z_2z0 = u_over_v_j.subs(z,-z+2*z0).subs([
    (sigma(-z+mu[j], g2,g3), -sigma(z-mu[j], g2,g3)),
    (sigma(-z + 2*z0 - mu[j], g2,g3), -sigma(z - 2*z0 + mu[j], g2,g3)),
    (sigma(-z+z0+z1, g2,g3), -sigma(z-z0-z1, g2,g3)),
    (sigma(-z + z0 - z1, g2,g3), -sigma(z - z0 + z1, g2,g3))
])
u_over_v_j_shifted_min_z_2z0_at_0 = u_over_v_j_shifted_min_z_2z0.subs(z,0)
u_v_reflection = Eq(
    u_over_v_j_shifted_min_z_2z0.lhs*u_over_v_j.lhs,
    (u_over_v_j_shifted_min_z_2z0.rhs*u_over_v_j.rhs)
    .subs(
        log(sigma(z-z0-z1, g2, g3)/sigma(z-z0 + z1, g2, g3)), 
        -log(sigma(z-z0 + z1, g2, g3)/sigma(z-z0-z1, g2, g3))
    )
    .expand().simplify()
).subs(z,z+z0)
u_v_reflection_at_0 = u_v_reflection.subs(z,0)
integer_2r1 = Eq(
    u_over_v_j_at_z0.lhs**2/u_v_reflection_at_0.lhs, 
    (u_over_v_j_at_z0.rhs**2/u_v_reflection_at_0.rhs).simplify()
)

u_over_v_j_at_0
u_over_v_j_at_z0
u_over_v_j_shifted_min_z_2z0
u_v_reflection
u_v_reflection_at_0
integer_2r1

Eq(u(0, mu[j])/v(0, mu[j]), sigma(-2*z0 + mu[j], g2, g3)*exp(2*log(sigma(-z0 + z1, g2, g3)/sigma(-z0 - z1, g2, g3))*r[1, j] + 2*epsilon[j])/sigma(-mu[j], g2, g3))

Eq(u(z0, mu[j])/v(z0, mu[j]), -exp(2*z0*r[0, j] + 2*epsilon[j] + 2*I*pi*r[1, j]))

Eq(u(-z + 2*z0, mu[j])/v(-z + 2*z0, mu[j]), sigma(z - mu[j], g2, g3)*exp(2*(-z + 2*z0)*r[0, j] + 2*log(sigma(z - z0 - z1, g2, g3)/sigma(z - z0 + z1, g2, g3))*r[1, j] + 2*epsilon[j])/sigma(z - 2*z0 + mu[j], g2, g3))

Eq(u(-z + z0, mu[j])*u(z + z0, mu[j])/(v(-z + z0, mu[j])*v(z + z0, mu[j])), exp(4*z0*r[0, j] + 4*epsilon[j]))

Eq(u(z0, mu[j])**2/v(z0, mu[j])**2, exp(4*z0*r[0, j] + 4*epsilon[j]))

Eq(1, exp(4*I*pi*r[1, j]))

It is likely that the funciton is undefined at $z=z_0$ as there is a discontinuity in the log phase term. It tends to different things when approaching from $z_0+\epsilon$ and $z_0-\epsilon$ (here $\epsilon$ is temporarily repurposed to mean a small perturbation):

In [52]:
log_term_discontinuous = log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))
log_term_discontinuous.subs(z,z0+epsilon)
log_term_discontinuous.subs(z,z0-epsilon).subs([
    (sigma(-epsilon-z1, g2, g3), -sigma(epsilon+z1, g2, g3)),
    (sigma(-epsilon+z1, g2, g3), -sigma(epsilon-z1, g2, g3))
])

log(sigma(z1 + epsilon, g2, g3)/sigma(-z1 + epsilon, g2, g3))

log(sigma(-z1 + epsilon, g2, g3)/sigma(z1 + epsilon, g2, g3))

## The distribution of the shared nonlinear phase term

In [53]:
r1j_a = r1j_log_part.subs([
    d_j_coeffs[4].subs([b_j_coeffs[2].args, c_j_coeffs[4].args]).args,
    Lams[1].subs(m,j).args
])
r1j_log_part
r1j_a

Eq(r[1, j], Lambda[1, j]/sqrt(d[4]))

Eq(r[1, j], (Sum(a[j, k], (k, 1, 4)) - Sum(a[j, k], (j, 1, 4), (k, 1, 4))/4)/sqrt(Sum(a[j, k]/2, (j, 1, 4), (k, 1, 4))**2 - 4))

## Case when a mode is zero at $z=0$

If $u(0,\mu_j)=v(0,\mu_j)=0$ for any $j$, then $\rho'(0)=0$ and so we may choose $\lambda_1=0$

In [54]:
lam1_zero = Eq(lam[1],0)

drho_uv_init
rho_lam
rho_q_lam1
qz_lam
lam1_zero

Eq(Subs(Derivative(rho(z), z), z, 0), u(0, mu[1])*u(0, mu[2])*u(0, mu[3])*u(0, mu[4]) - v(0, mu[1])*v(0, mu[2])*v(0, mu[3])*v(0, mu[4]))

Eq(Derivative(rho(z), z)**2, d[4]*Product(rho(z) - lambda[j], (j, 1, 4)))

Eq(rho(z), q(z) + lambda[1])

Eq(Derivative(q(z), z)**2, (q(z) + lambda[1] - lambda[2])*(q(z) + lambda[1] - lambda[3])*(q(z) + lambda[1] - lambda[4])*q(z)*d[4])

Eq(lambda[1], 0)

In [55]:
d_j_coeffs_wp_with_lam1_zero = [_eq.subs(*lam1_zero.args) for _eq in d_j_coeffs_wp]
lam_1_zero_subs = [
    lam1_zero.args,
    (wp(z1,g2,g3), solve(d_j_coeffs_wp_with_lam1_zero[2], wp(z1,g2,g3))[0]),
    (wpp(z1,g2,g3), solve(d_j_coeffs_wp_with_lam1_zero[1], wpp(z1,g2,g3))[0]),
    (diff(rho(z),z).subs(z,0).doit(),0)
]


for _eq in d_j_coeffs_wp_with_lam1_zero:
    _eq
    
rho_z_sqrt_d4.subs(lam_1_zero_subs)

Eq(d[0], 0)

Eq(4*\wp\'(z1, g2, g3)/sqrt(d[4]), -d[1])

Eq(12*wp(z1, g2, g3), d[2])

Eq(d[3], 4*(g2/4 - 3*wp(z1, g2, g3)**2)*sqrt(d[4])/\wp\'(z1, g2, g3))

Eq(rho(z), -d[1]/(4*(-wp(z - z0, g2, g3) + d[2]/12)))

The point $z_0$ is a half-period:

In [56]:
z02_sigma_shift = (
    quasi_period_sigma_b
    .subs(omega[1], solve(Eq(-2*z0, 2*m*omega[3] + 2*n*omega[1]),omega[1])[0])
    .subs(zeta(-z0, g2, g3), -zeta(z0, g2, g3))
)

wp_z0_rho0_lam1.subs(lam_1_zero_subs)
wpp_z0_rho0_lam1.subs(lam_1_zero_subs)
z02_sigma_shift

Eq(wp(z0, g2, g3), d[2]/12 + d[1]/(4*rho(0)))

Eq(\wp\'(z0, g2, g3), 0)

Eq(sigma(z - 2*z0, g2, g3), (-1)**(m*n + m + n)*sigma(z, g2, g3)*exp(-(2*z - 2*z0)*zeta(z0, g2, g3)))

If as $z \rightarrow 0$ we also have $u(z,\mu_j) \rightarrow 0$ and $v(z,\mu_j) \rightarrow 0$ but their ratio tends to a constant, then we can use that limit to find $\epsilon_j$:

In [57]:
u_over_v_j

Eq(u(z, mu[j])/v(z, mu[j]), sigma(z - 2*z0 + mu[j], g2, g3)*exp(2*z*r[0, j] + 2*log(sigma(z - z0 + z1, g2, g3)/sigma(z - z0 - z1, g2, g3))*r[1, j] + 2*epsilon[j])/sigma(z - mu[j], g2, g3))

In [58]:
u_over_v_j_z0_uv_zero = u_over_v_j.subs(z,0).subs([
    z02_sigma_shift.subs(z,mu[j]).args,
    (sigma(-mu[j], g2, g3), -sigma(mu[j], g2, g3)),
    (sigma(-z0-z1, g2, g3), -sigma(z0+z1, g2, g3)),
    z02_sigma_shift.subs(z,z1+z0).args,
#     (-(-1)**(m*n+m+n), exp(I*pi*(m*n+m+n+1))),
#     (
#         log(exp(-2*z1*zeta(z0, g2, g3))*exp(I*pi*(m*n + m + n + 1))),
#         -2*z1*zeta(z0, g2, g3)+ I*pi*(m*n + m + n + 1)
#     )
]).simplify()
u_over_v_j_z0_uv_zero

Eq(u(0, mu[j])/v(0, mu[j]), (-1)**(m*n + m + n + 1)*exp(2*(z0 - mu[j])*zeta(z0, g2, g3) + 2*log((-1)**(m*n + m + n + 1)*exp(-2*z1*zeta(z0, g2, g3)))*r[1, j] + 2*epsilon[j]))

## The case where the quartic $\rho$ term cancels to give a cubic

There will be no $\rho(z)^4$ term in the differential equation when $d_4=0$. This case is simpler to solve and leads to the cases for polarization dynamics and the PT symmetric examples in cubic nonlinearity that were considered in my previous paper [General complex envelope solutions of coupled-mode optics with quadratic or cubic nonlinearity](https://arxiv.org/abs/1512.03092).


For $d_4=0$ we require $b_2 \pm 2$, which is a requirement of the parameters, i.e., it cannot be achieved in general through tailored initial conditions of the modes.

In [59]:
drhop_b
drhop_d

Eq(Derivative(rho(z), z)**2, (rho(z)**2*b[2] + rho(z)*b[1] + b[0])**2 - 4*Product(-rho(z) + gamma[j], (j, 1, 4)))

Eq(Derivative(rho(z), z)**2, Sum(rho(z)**j*d[j], (j, 0, 4)))

In [60]:
no_4th_d_zero = d_j_coeffs[4].subs([_eq.args for _eq in c_j_coeffs]).subs(d[4],0)
no_4th_d_zero_b2_sols = [Eq(b[2], _s) for _s in solve(no_4th_d_zero, b[2])]

c_j_coeffs[4]
d_j_coeffs[4]
no_4th_d_zero
for _eq in no_4th_d_zero_b2_sols:
    _eq
b_j_coeffs[2]

Eq(c[4], 1)

Eq(d[4], b[2]**2 - 4*c[4])

Eq(0, b[2]**2 - 4)

Eq(b[2], -2)

Eq(b[2], 2)

Eq(b[2], Sum(a[j, k]/2, (j, 1, 4), (k, 1, 4)))

When $d_4 \rightarrow 0$, $\wp(z_1,g_2,g_3)=0$ thus $z_1=\omega_i$:

In [61]:
wpp_z1_d = Eq(wpp(z1,g2,g3), solve(d_j_coeffs_wp[1], wpp(z1,g2,g3))[0])
wp_z1_d =Eq(wp(z1,g2,g3), solve(d_j_coeffs_wp[2], wp(z1,g2,g3))[0])
wp_z1_is_omega_i = Eq(z1,omega[i])
wpp_z1_d_2 = Eq(wpp(z1,g2,g3), 0)

wpp_z1_d
wpp_z1_d_2
wp_z1_is_omega_i
wp_z1_d

Eq(\wp\'(z1, g2, g3), (-d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)*sqrt(d[4])/4)

Eq(\wp\'(z1, g2, g3), 0)

Eq(z1, omega[i])

Eq(wp(z1, g2, g3), d[2]/12 + d[3]*lambda[1]/4 + d[4]*lambda[1]**2/2)

Under those conditions in which $d_4 \rightarrow 0, z_1=\omega_i$, the $\wp$ function can be moved from the denominator to the numerator using an identity of $\wp$:

In [62]:
rho_z_no_sqrt_d4_b = rho_z_sqrt_d4.subs(wpp(z1,g2,g3), solve(d_j_coeffs_wp[3], wpp(z1,g2,g3))[0])

rho_z_no_sqrt_d4_c = rho_z_no_sqrt_d4_b.subs(d[4],0).subs(
    lam[1], 
    solve(rho_z_no_sqrt_d4_b.subs(d[4],0).subs([(z,0), (wp(-z0,g2,g3), wp(z0,g2,g3))]), lam[1])[0]
    .expand().collect(rho(0), factor)
)
pw_addition_id_omega_g2_z1_z0 = pw_addition_id_omega_g2.subs([(z,z-z0), (omega[i],z1)]).simplify()
rho_z_no_sqrt_d4_d = Eq(
    rho_z_no_sqrt_d4_c.lhs,
    (
        rho_z_no_sqrt_d4_c.rhs 
        + 4*pw_addition_id_omega_g2_z1_z0.rhs/d[3] 
        - 4*pw_addition_id_omega_g2_z1_z0.lhs/d[3]
        - 4*pw_addition_id_omega_g2_z1_z0.rhs.subs(z,0)/d[3] 
        + 4*pw_addition_id_omega_g2_z1_z0.lhs.subs(z,0)/d[3]
    ).subs(wp(-z0,g2,g3), wp(z0,g2,g3))
    .expand().collect([g2, wp(z1, g2,g3)**2], simplify).expand()
)

rho_z_no_sqrt_d4_b
rho_z_no_sqrt_d4_c
pw_addition_id_omega_g2
rho_z_no_sqrt_d4_d

Eq(rho(z), (g2 - 12*wp(z1, g2, g3)**2)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))*(d[3] + 4*d[4]*lambda[1])) + lambda[1])

Eq(rho(z), (g2 - 12*wp(z1, g2, g3)**2)/((wp(z1, g2, g3) - wp(z - z0, g2, g3))*d[3]) + (g2 - 12*wp(z1, g2, g3)**2)/((wp(z0, g2, g3) - wp(z1, g2, g3))*d[3]) + rho(0))

Eq(wp(z + omega[i], g2, g3) - wp(omega[i], g2, g3), (-g2/4 + 3*wp(omega[i], g2, g3)**2)/(wp(z, g2, g3) - wp(omega[i], g2, g3)))

Eq(rho(z), rho(0) - 4*wp(-z0 + z1, g2, g3)/d[3] + 4*wp(z - z0 + z1, g2, g3)/d[3])

In contrast, solving the cubic differential equation directly gives:

In [63]:
drho_no_d4 = drhop_d.doit().subs(d[4],0)
rho_cubic_no_d4_w_sub = Eq(rho(z),4*w(z)/d[3] - d[2]/d[3]/3)

w_cubic_no_d4_lim = Eq(
    diff(w(z),z)**2, 
    solve(drho_no_d4.subs(*rho_cubic_no_d4_w_sub.args).doit(), diff(w(z),z)**2)[0]
    .expand().collect(w(z), factor)
)

rho_z_d4_zero_lim_cubic = Eq(rho(z),((wp(z-x,g2,g3) - d[2]/12)*4/d[3]).expand())
x_z1_z0_choice = Eq(x, z0-z1)
rho_z_d4_zero_lim_cubic_b = rho_z_d4_zero_lim_cubic.subs(*x_z1_z0_choice.args)

drhop_b
drhop_d

drho_no_d4
rho_cubic_no_d4_w_sub
w_cubic_no_d4_lim
rho_z_d4_zero_lim_cubic
x_z1_z0_choice
rho_z_d4_zero_lim_cubic_b

Eq(Derivative(rho(z), z)**2, (rho(z)**2*b[2] + rho(z)*b[1] + b[0])**2 - 4*Product(-rho(z) + gamma[j], (j, 1, 4)))

Eq(Derivative(rho(z), z)**2, Sum(rho(z)**j*d[j], (j, 0, 4)))

Eq(Derivative(rho(z), z)**2, rho(z)**3*d[3] + rho(z)**2*d[2] + rho(z)*d[1] + d[0])

Eq(rho(z), 4*w(z)/d[3] - d[2]/(3*d[3]))

Eq(Derivative(w(z), z)**2, (3*d[1]*d[3] - d[2]**2)*w(z)/12 + (27*d[0]*d[3]**2 - 9*d[1]*d[2]*d[3] + 2*d[2]**3)/432 + 4*w(z)**3)

Eq(rho(z), 4*wp(-x + z, g2, g3)/d[3] - d[2]/(3*d[3]))

Eq(x, z0 - z1)

Eq(rho(z), 4*wp(z - z0 + z1, g2, g3)/d[3] - d[2]/(3*d[3]))

which, assuming $x=z_0-z_1$, would imply that:

In [64]:
rho_0_d2_d3_z0_z1 = Eq(
    rho_z_no_sqrt_d4_d.lhs - rho_z_d4_zero_lim_cubic.lhs,
    (rho_z_no_sqrt_d4_d.rhs - rho_z_d4_zero_lim_cubic.rhs).subs(x,z0-z1)
)
rho_0_d2_d3_z0_z1

Eq(0, rho(0) - 4*wp(-z0 + z1, g2, g3)/d[3] + d[2]/(3*d[3]))

We can check this as follows. From either formula we know that:

In [65]:
drho_0_d2_d3_z0_z1 = Eq(
    diff(rho_z_no_sqrt_d4_d.lhs, z).subs(z,0),
    diff(rho_z_no_sqrt_d4_d.rhs, z)
    .subs(z,0)
    .replace(diff(wp(z-z0+z1,g2,g3),z).subs(z,0).doit(), wpp(-z0+z1, g2, g3))
)
drho_0_d2_d3_z0_z1

Eq(Subs(Derivative(rho(z), z), z, 0), 4*\wp\'(-z0 + z1, g2, g3)/d[3])

and by combining this with $\wp, \wp'$ addition identities at the point $z_0-z_1$ we verify the relation:

In [66]:
wp_z1_min_z0 = pw_addition_id.subs([(x,z1), (y,-z0)]).subs([
    (wp(-z0,g2,g3), wp(z0,g2,g3)), 
    (wpp(-z0,g2,g3), -wpp(z0,g2,g3)),
    wp_z0_rho0_lam1.args,
    wpp_z0_rho0_lam1.args,
    wp_z1_d.args,
    wpp_z1_d.args 
]).expand().simplify()
wp_z1_min_z0 = Eq(wp_z1_min_z0.lhs, wp_z1_min_z0.rhs.expand().collect([diff(rho(z),z).subs(z,0), rho(0)], factor))
wp_z1_min_z0_no_d4 = wp_z1_min_z0.subs(d[4],0)

wpp_z1_min_z0 = pwp_xy_addition.subs([(x,z1), (y,-z0)]).subs([
    (wp(-z0,g2,g3), wp(z0,g2,g3)), 
    (wpp(-z0,g2,g3), -wpp(z0,g2,g3)),
    wp_z0_rho0_lam1.args,
    wpp_z0_rho0_lam1.args,
    wp_z1_d.args,
    wpp_z1_d.args 
])
wpp_z1_min_z0_no_d4 = wpp_z1_min_z0.subs(d[4],0)
wpp_z1_min_z0_no_d4 = Eq(
    wpp_z1_min_z0_no_d4.lhs, 
    wpp_z1_min_z0_no_d4.rhs.expand().collect(diff(rho(z),z).subs(z,0), factor)
)
drho0_sqrd = Eq(
    diff(rho(z),z).subs(z,0)**2,
    solve(
        wpp_z1_min_z0_no_d4.lhs/drho_0_d2_d3_z0_z1.rhs - 
        (wpp_z1_min_z0_no_d4.rhs/drho_0_d2_d3_z0_z1.lhs).simplify(),
        diff(rho(z),z).subs(z,0)**2
    )[0]
)

wp_z1_min_z0_no_d4_b = wp_z1_min_z0_no_d4.subs([drho0_sqrd.args]).simplify()

wp_z0_rho0_lam1
wpp_z0_rho0_lam1
d_j_coeffs_wp[1]
d_j_coeffs_wp[2]

wp_z1_min_z0
wp_z1_min_z0_no_d4
wpp_z1_min_z0_no_d4
drho0_sqrd
wp_z1_min_z0_no_d4_b

if not wp_z1_min_z0_no_d4_b.subs(rho(0), solve(rho_0_d2_d3_z0_z1.args, rho(0))[rho(0)]).simplify():
    raise ValueError('wp_z1_min_z0_no_d4_b check failed')

Eq(wp(z0, g2, g3), d[2]/12 + d[3]*lambda[1]/4 + d[4]*lambda[1]**2/2 + (-d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)/(4*(-rho(0) + lambda[1])))

Eq(\wp\'(z0, g2, g3), (d[1] + 2*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 4*d[4]*lambda[1]**3)*Subs(Derivative(rho(z), z), z, 0)/(4*(rho(0) - lambda[1])**2))

Eq(4*\wp\'(z1, g2, g3)/sqrt(d[4]), -d[1] - 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2 - 4*d[4]*lambda[1]**3)

Eq(12*wp(z1, g2, g3), d[2] + 3*d[3]*lambda[1] + 6*d[4]*lambda[1]**2)

Eq(wp(-z0 + z1, g2, g3), -sqrt(d[4])*Subs(Derivative(rho(z), z), z, 0)/2 - (3*d[1] + 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2)*rho(0)/(12*(rho(0) - lambda[1])**2) - (d[2] + 3*d[3]*lambda[1] - 3*d[4]*lambda[1]**2)*rho(0)**2/(6*(rho(0) - lambda[1])**2) + (3*d[1] + 4*d[2]*lambda[1] + 3*d[3]*lambda[1]**2 + 3*d[4]*lambda[1]**3)*lambda[1]/(12*(rho(0) - lambda[1])**2) + rho(0)**4*d[4]/(4*(rho(0) - lambda[1])**2) - rho(0)**3*d[4]*lambda[1]/(rho(0) - lambda[1])**2 + Subs(Derivative(rho(z), z), z, 0)**2/(4*(rho(0) - lambda[1])**2))

Eq(wp(-z0 + z1, g2, g3), -(d[2] + 3*d[3]*lambda[1])*rho(0)**2/(6*(rho(0) - lambda[1])**2) - (3*d[1] + 2*d[2]*lambda[1] - 3*d[3]*lambda[1]**2)*rho(0)/(12*(rho(0) - lambda[1])**2) + (3*d[1] + 4*d[2]*lambda[1] + 3*d[3]*lambda[1]**2)*lambda[1]/(12*(rho(0) - lambda[1])**2) + Subs(Derivative(rho(z), z), z, 0)**2/(4*(rho(0) - lambda[1])**2))

Eq(\wp\'(-z0 + z1, g2, g3), -(rho(0)*d[2] + 3*rho(0)*d[3]*lambda[1] + d[1] + d[2]*lambda[1])*Subs(Derivative(rho(z), z), z, 0)/(4*(-rho(0) + lambda[1])**2) - Subs(Derivative(rho(z), z), z, 0)**3/(4*(-rho(0) + lambda[1])**3))

Eq(Subs(Derivative(rho(z), z), z, 0)**2, rho(0)**3*d[3] + rho(0)**2*d[2] + rho(0)*d[1] - d[1]*lambda[1] - d[2]*lambda[1]**2 - d[3]*lambda[1]**3)

Eq(wp(-z0 + z1, g2, g3), rho(0)*d[3]/4 + d[2]/12)

When integrating the gauge function, we note that previously when $d_4 \ne 0$ we had:

In [67]:
dphi_m_Q_Lam.subs([(m,j), rho_zeta.args])

Eq(Derivative(phi(z, mu[j]), z), (-(2*zeta(z1, g2, g3) - zeta(-z + z0 + z1, g2, g3) - zeta(z - z0 + z1, g2, g3))/sqrt(d[4]) + lambda[1])*Lambda[1, j] + Lambda[0, j])

but as $d_4 \rightarrow 0$, this is replaced with:


In [68]:
wp_as_zeta_diff_z0_z1 = Eq(wp(z-z0+z1,g2,g3), -Derivative(zeta(z-z0+z1,g2,g3),z))
rho_z_d4_zero_zeta_diff = rho_z_d4_zero_lim_cubic.subs(x,z0-z1).subs([wp_as_zeta_diff_z0_z1.args])
dphi_m_Q_Lam_cubic = dphi_m_Q_Lam.subs([(m,j), rho_z_d4_zero_zeta_diff.args])

wp_as_zeta_diff_z0_z1
rho_z_d4_zero_zeta_diff
dphi_m_Q_Lam_cubic

Eq(wp(z - z0 + z1, g2, g3), -Derivative(zeta(z - z0 + z1, g2, g3), z))

Eq(rho(z), -4*Derivative(zeta(z - z0 + z1, g2, g3), z)/d[3] - d[2]/(3*d[3]))

Eq(Derivative(phi(z, mu[j]), z), (-4*Derivative(zeta(z - z0 + z1, g2, g3), z)/d[3] - d[2]/(3*d[3]))*Lambda[1, j] + Lambda[0, j])

In [69]:
dlog_z_plus_x_cubic = Eq(
    ((Derivative(rho(x), x) - Derivative(rho(z), z))/(2*(rho(x) - rho(z))))
    .subs([
        (diff(rho(z),z),rhop(z)), (diff(rho(x),x),rhop(x))
    ])
    .subs(x, mu[j])
    .simplify(),
    ((Derivative(rho(x), x) - Derivative(rho(z), z))/(2*(rho(x) - rho(z))))
    .subs([rho_z_d4_zero_lim_cubic_b.args, rho_z_d4_zero_lim_cubic_b.subs(z,x).args])
    .doit()
    .replace(diff(wp(x,g2,g3),x).subs(x,z-z0+z1).doit(), wpp(z-z0+z1, g2, g3))
    .replace(diff(wp(z,g2,g3),z).subs(z,x-z0+z1).doit(), wpp(x-z0+z1, g2, g3))
    .simplify()
    .subs(x, mu[j])
)

dlog_z_min_x_cubic = Eq(
    ((-Derivative(rho(x), x) - Derivative(rho(z), z))/(2*(rho(x) - rho(z))))
    .subs([
        (diff(rho(z),z),rhop(z)), (diff(rho(x),x),rhop(x))
    ])
    .subs(x, mu[j])
    .simplify(),
    ((-Derivative(rho(x), x) - Derivative(rho(z), z))/(2*(rho(x) - rho(z))))
    .subs([rho_z_d4_zero_lim_cubic_b.args, rho_z_d4_zero_lim_cubic_b.subs(z,x).args])
    .doit()
    .replace(diff(wp(x,g2,g3),x).subs(x,z-z0+z1).doit(), wpp(z-z0+z1, g2, g3))
    .replace(diff(wp(z,g2,g3),z).subs(z,x-z0+z1).doit(), wpp(x-z0+z1, g2, g3))
    .simplify()
    .subs(x, mu[j])
)

pw_to_zw_ids_cubic = [
    pw_to_zw_identity
    .subs([(z,t),(x,y)]).subs([(t,z-z0+z1), (y, -z0+z1+mu[j])]),
    pw_to_zw_identity.subs([(x,-x),(wp(-x,g2,g3),wp(x,g2,g3)),(wpp(-x,g2,g3),-wpp(x,g2,g3))])
    .subs([(z,t),(x,y)]).subs([(t,z-z0+z1), (y, -z0+z1+mu[j])])
]

dlog_z_plus_x_zeta_cubic = Eq(
    dlog_z_plus_x_cubic.lhs,
    (dlog_z_plus_x_cubic.rhs - pw_to_zw_ids_cubic[0].lhs + pw_to_zw_ids_cubic[0].rhs)
    .collect(pw_to_zw_ids_cubic[0].rhs, simplify)
)
dlog_z_min_x_zeta_cubic = Eq(
    dlog_z_min_x_cubic.lhs,
    (dlog_z_min_x_cubic.rhs - pw_to_zw_ids_cubic[1].lhs + pw_to_zw_ids_cubic[1].rhs)
    .collect(pw_to_zw_ids_cubic[1].rhs, simplify)
)

dlog_z_plus_x_cubic
dlog_z_min_x_cubic
dlog_z_plus_x_zeta_cubic
dlog_z_min_x_zeta_cubic

Eq((\\rho\'(z) - \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))), (-\wp\'(z - z0 + z1, g2, g3) + \wp\'(-z0 + z1 + mu[j], g2, g3))/(2*(-wp(z - z0 + z1, g2, g3) + wp(-z0 + z1 + mu[j], g2, g3))))

Eq((\\rho\'(z) + \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))), (-\wp\'(z - z0 + z1, g2, g3) - \wp\'(-z0 + z1 + mu[j], g2, g3))/(2*(-wp(z - z0 + z1, g2, g3) + wp(-z0 + z1 + mu[j], g2, g3))))

Eq((\\rho\'(z) - \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))), -zeta(z - z0 + z1, g2, g3) - zeta(-z0 + z1 + mu[j], g2, g3) + zeta(z - 2*z0 + 2*z1 + mu[j], g2, g3))

Eq((\\rho\'(z) + \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))), zeta(z - mu[j], g2, g3) - zeta(z - z0 + z1, g2, g3) - zeta(z0 - z1 - mu[j], g2, g3))

In [70]:
dvz_unified.subs(1/(2*rho(z) - 2*rho(mu[j])), 1/(rho(z) - rho(mu[j]))/2)

dlog_z_min_x_zeta_cubic
dlog_z_plus_x_zeta_cubic

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]), (\\rho\'(z) + \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))) - Derivative(phi(z, mu[j]), z))

Eq((\\rho\'(z) + \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))), zeta(z - mu[j], g2, g3) - zeta(z - z0 + z1, g2, g3) - zeta(z0 - z1 - mu[j], g2, g3))

Eq((\\rho\'(z) - \\rho\'(mu[j]))/(2*(rho(z) - rho(mu[j]))), -zeta(z - z0 + z1, g2, g3) - zeta(-z0 + z1 + mu[j], g2, g3) + zeta(z - 2*z0 + 2*z1 + mu[j], g2, g3))

In [71]:
## -- Build zeta expansion for log diffs of u, v --
cubic_rho_phi_subs = [
    (1/(2*rho(z) - 2*rho(mu[j])), 1/(rho(z) - rho(mu[j]))/2),
    dphi_m_Q_Lam_cubic.args
]
duz_unified_zeta_cubic = duz_unified.subs(cubic_rho_phi_subs + [dlog_z_plus_x_zeta_cubic.args])
dvz_unified_zeta_cubic = dvz_unified.subs(cubic_rho_phi_subs + [dlog_z_min_x_zeta_cubic.args])

## -- Convert zetas to log diff sigmas --
zw_dlog_subs_cubic = [
    zw_dlog_sigma_identity.subs([(x, mu[j])]).args,
    zw_dlog_sigma_identity.subs([(x, z0 - z1)]).args,
    zw_dlog_sigma_identity.subs([(x, 2*z0 - 2*z1 - mu[j])]).args
]

duz_unified_zeta_cubic_b = Eq(
    duz_unified_zeta_cubic.lhs,
    (
        duz_unified_zeta_cubic.rhs 
        - zw_dlog_subs_cubic[1][1] + zw_dlog_subs_cubic[1][0]
        + zw_dlog_subs_cubic[2][1] - zw_dlog_subs_cubic[2][0]
    ).subs(zeta(-z0 + z1 + mu[j], g2, g3), -zeta(z0 - z1 - mu[j], g2, g3))
)
dvz_unified_zeta_cubic_b = Eq(
    dvz_unified_zeta_cubic.lhs,
    dvz_unified_zeta_cubic.rhs 
    - zw_dlog_subs_cubic[1][1] + zw_dlog_subs_cubic[1][0]
    + zw_dlog_subs_cubic[0][1] - zw_dlog_subs_cubic[0][0]
)

## -- Combine log diff sigmas --

cubic_log_diff_subs = [
    Eq(
        - Derivative(log(sigma(z - z0 + z1, g2, g3)), z) 
        + Derivative(log(sigma(z - 2*z0 + 2*z1 + mu[j], g2, g3)), z),
        Derivative(log(sigma(z - 2*z0 + 2*z1 + mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)), z)
    ),
    Eq(
        Derivative(log(sigma(z - mu[j], g2, g3)), z) - 
        Derivative(log(sigma(z - z0 + z1, g2, g3)), z),
        Derivative(log(sigma(z - mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)), z)
    )
]
duz_unified_zeta_cubic_c = duz_unified_zeta_cubic_b.subs(*cubic_log_diff_subs[0].args)
dvz_unified_zeta_cubic_c = dvz_unified_zeta_cubic_b.subs(*cubic_log_diff_subs[1].args)


## -- Define the solutions after integrating --

uz_sol_cubic = Eq(
    u(z, mu[j]),
    sigma(z - 2*z0 + 2*z1 + mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)*
    exp(
        -4*Lam[1, j]*zeta(z - z0 + z1, g2, g3)/d[3] + 
        (zeta(z0 - z1 - mu[j], g2, g3) + d[2]*Lam[1, j]/(3*d[3]) + Lam[0, j])*z
        + r[3,j]/2 + epsilon[j]
    )
)
vz_sol_cubic = Eq(
    v(z, mu[j]),
    sigma(z - mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)*
    exp(
        4*Lam[1, j]*zeta(z - z0 + z1, g2, g3)/d[3] - 
        (zeta(z0 - z1 - mu[j], g2, g3) + d[2]*Lam[1, j]/(3*d[3]) + Lam[0, j])*z
        + r[3,j]/2 - epsilon[j]
    )
)


duz_unified_zeta_cubic
dvz_unified_zeta_cubic

duz_unified_zeta_cubic_c
dvz_unified_zeta_cubic_c

uz_sol_cubic
vz_sol_cubic

Eq(Derivative(u(z, mu[j]), z)/u(z, mu[j]), (-4*Derivative(zeta(z - z0 + z1, g2, g3), z)/d[3] - d[2]/(3*d[3]))*Lambda[1, j] - zeta(z - z0 + z1, g2, g3) - zeta(-z0 + z1 + mu[j], g2, g3) + zeta(z - 2*z0 + 2*z1 + mu[j], g2, g3) + Lambda[0, j])

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]), -(-4*Derivative(zeta(z - z0 + z1, g2, g3), z)/d[3] - d[2]/(3*d[3]))*Lambda[1, j] + zeta(z - mu[j], g2, g3) - zeta(z - z0 + z1, g2, g3) - zeta(z0 - z1 - mu[j], g2, g3) - Lambda[0, j])

Eq(Derivative(u(z, mu[j]), z)/u(z, mu[j]), (-4*Derivative(zeta(z - z0 + z1, g2, g3), z)/d[3] - d[2]/(3*d[3]))*Lambda[1, j] + zeta(z0 - z1 - mu[j], g2, g3) + Derivative(log(sigma(z - 2*z0 + 2*z1 + mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)), z) + Lambda[0, j])

Eq(Derivative(v(z, mu[j]), z)/v(z, mu[j]), -(-4*Derivative(zeta(z - z0 + z1, g2, g3), z)/d[3] - d[2]/(3*d[3]))*Lambda[1, j] - zeta(z0 - z1 - mu[j], g2, g3) + Derivative(log(sigma(z - mu[j], g2, g3)/sigma(z - z0 + z1, g2, g3)), z) - Lambda[0, j])

Eq(u(z, mu[j]), sigma(z - 2*z0 + 2*z1 + mu[j], g2, g3)*exp(z*(zeta(z0 - z1 - mu[j], g2, g3) + Lambda[0, j] + Lambda[1, j]*d[2]/(3*d[3])) - 4*zeta(z - z0 + z1, g2, g3)*Lambda[1, j]/d[3] + epsilon[j] + r[3, j]/2)/sigma(z - z0 + z1, g2, g3))

Eq(v(z, mu[j]), sigma(z - mu[j], g2, g3)*exp(-z*(zeta(z0 - z1 - mu[j], g2, g3) + Lambda[0, j] + Lambda[1, j]*d[2]/(3*d[3])) + 4*zeta(z - z0 + z1, g2, g3)*Lambda[1, j]/d[3] - epsilon[j] + r[3, j]/2)/sigma(z - z0 + z1, g2, g3))

The $r_{3,j}$ parameter can be obtained by comparing to the expression $\rho$ in the cubic case:

In [72]:
uv_sol_cubic = Eq(
    uz_sol_cubic.lhs*vz_sol_cubic.lhs, 
    (uz_sol_cubic.rhs*vz_sol_cubic.rhs).simplify()
)
uv_rho_gam = Eq(u(z, mu[j])*v(z, mu[j]), gamma[j] - rho(z))
uv_sol_cubic_b = Eq(
    (uv_sol_cubic.lhs - uv_sol_cubic.lhs.subs(z,0))
    .subs(
        uv_rho_gam.lhs - uv_rho_gam.lhs.subs(z,0), 
        uv_rho_gam.rhs - uv_rho_gam.rhs.subs(z,0)
    )
    .subs(*rho_z_no_sqrt_d4_d.args),
    (uv_sol_cubic.rhs - uv_sol_cubic.rhs.subs(z,0))
    .subs([
        (
            sigma(z-mu[j], g2,g3), 
            solve(sigma_p_identity.subs([(x, z - z0 + z1), (y, mu[j] - z0 + z1)]), sigma(z-mu[j], g2,g3))[0]
        )
    ]),
)
cubic_exp_r3j = Eq(
    uv_sol_cubic_b.lhs.coeff(wp(z - z0+ z1,g2,g3)),
    uv_sol_cubic_b.rhs.expand().coeff(wp(z - z0+ z1,g2,g3))
)
cubic_exp_r3j = Eq(exp(r[3,j]), solve(cubic_exp_r3j, exp(r[3,j]))[0])

uv_sol_cubic_c = uv_sol_cubic_b.subs([cubic_exp_r3j.args])
uv_sol_cubic_check = Eq(
    ((uv_sol_cubic_c.lhs - uv_sol_cubic_c.rhs)*d[3]/4).expand()
    .subs(*sigma_p_identity.subs([(x, z - z0 + z1), (y, mu[j] - z0 + z1)]).subs(z,0).args),
    0
)
if not uv_sol_cubic_check:
    raise ValueError('uv_sol_cubic_check failed')

cubic_exp_r3j_over2 = Eq(exp(r[3, j]/2), 2/(sigma(-z0 + z1 + mu[j], g2, g3)*sqrt(d[3])))
if not (cubic_exp_r3j.rhs - cubic_exp_r3j_over2.rhs**2) == 0:
    raise ValueError('uv_sol_cubic_check2 failed')


cubic_exp_r3j
cubic_exp_r3j_over2

Eq(exp(r[3, j]), 4/(sigma(-z0 + z1 + mu[j], g2, g3)**2*d[3]))

Eq(exp(r[3, j]/2), 2/(sigma(-z0 + z1 + mu[j], g2, g3)*sqrt(d[3])))

Therefore, the solutions in the cubic case become:

In [73]:
uz_sol_cubic_b = Eq(uz_sol_cubic.lhs, uz_sol_cubic.rhs.subs(r[3,j],0)*cubic_exp_r3j_over2.rhs)
vz_sol_cubic_b = Eq(vz_sol_cubic.lhs, vz_sol_cubic.rhs.subs(r[3,j],0)*cubic_exp_r3j_over2.rhs)
uz_sol_cubic_b
vz_sol_cubic_b

Eq(u(z, mu[j]), 2*sigma(z - 2*z0 + 2*z1 + mu[j], g2, g3)*exp(z*(zeta(z0 - z1 - mu[j], g2, g3) + Lambda[0, j] + Lambda[1, j]*d[2]/(3*d[3])) - 4*zeta(z - z0 + z1, g2, g3)*Lambda[1, j]/d[3] + epsilon[j])/(sigma(z - z0 + z1, g2, g3)*sigma(-z0 + z1 + mu[j], g2, g3)*sqrt(d[3])))

Eq(v(z, mu[j]), 2*sigma(z - mu[j], g2, g3)*exp(-z*(zeta(z0 - z1 - mu[j], g2, g3) + Lambda[0, j] + Lambda[1, j]*d[2]/(3*d[3])) + 4*zeta(z - z0 + z1, g2, g3)*Lambda[1, j]/d[3] - epsilon[j])/(sigma(z - z0 + z1, g2, g3)*sigma(-z0 + z1 + mu[j], g2, g3)*sqrt(d[3])))

In [74]:
epsilon_j_cubic_from_u = Eq(epsilon[j], solve(uz_sol_cubic_b.subs(z,0), epsilon[j])[0])
epsilon_j_cubic_from_v = Eq(epsilon[j], solve(vz_sol_cubic_b.subs(z,0), epsilon[j])[0])
epsilon_u_over_v_cubic = Eq(
    epsilon[j], 
    solve(
        Eq(
            uz_sol_cubic_b.lhs.subs(z,0)/vz_sol_cubic_b.lhs.subs(z,0),
            uz_sol_cubic_b.rhs.subs(z,0)/vz_sol_cubic_b.rhs.subs(z,0)
        )
        , 2*epsilon[j]
    )[0]/2
)

epsilon_j_cubic_from_u
epsilon_j_cubic_from_v
epsilon_u_over_v_cubic

Eq(epsilon[j], log(sigma(-z0 + z1, g2, g3)*sigma(-z0 + z1 + mu[j], g2, g3)*u(0, mu[j])*exp(4*zeta(-z0 + z1, g2, g3)*Lambda[1, j]/d[3])*sqrt(d[3])/(2*sigma(-2*z0 + 2*z1 + mu[j], g2, g3))))

Eq(epsilon[j], log(2*sigma(-mu[j], g2, g3)*exp(4*zeta(-z0 + z1, g2, g3)*Lambda[1, j]/d[3])/(sigma(-z0 + z1, g2, g3)*sigma(-z0 + z1 + mu[j], g2, g3)*v(0, mu[j])*sqrt(d[3]))))

Eq(epsilon[j], log(sigma(-mu[j], g2, g3)*u(0, mu[j])*exp(8*zeta(-z0 + z1, g2, g3)*Lambda[1, j]/d[3])/(sigma(-2*z0 + 2*z1 + mu[j], g2, g3)*v(0, mu[j])))/2)